In [ ]:
%load_ext autoreload
%autoreload 2

## Set up environment
### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import random
import re
import scipy as sp
import seaborn as sns
import yaml
import shutil

from copy import deepcopy
from pathlib import Path
from collections import namedtuple, defaultdict
from typing import Literal

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rcParams

from assistive_arm.utils.data_preprocessing import read_headers, smooth_dataframe
from assistive_arm.utils.emg_processing import filter_emg, interpolate_dataframe_to_length
from assistive_arm.utils.emg_plotting import plot_every_muscle, plot_muscle_emg
from assistive_arm.utils.optimum_length_utils import get_jacobian, get_rotation_matrix
from assistive_arm.utils.printing import print_dict_structure


np.set_printoptions(precision=3, suppress=True)

### Plotting settings

In [ ]:
# Plot settings
plt.style.use('fivethirtyeight')

rcParams['font.size'] = 12

rcParams['figure.titlesize'] = 16
rcParams['figure.titleweight'] = 'normal'

rcParams['axes.labelsize'] = 14
rcParams['axes.titlesize'] = 16
rcParams['axes.titleweight'] = 'normal'

rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16

rcParams['axes.grid'] = False
rcParams['grid.linewidth'] = 0.5

# make background white
rcParams['figure.facecolor'] = 'white'
rcParams['axes.facecolor'] = 'white'
rcParams['savefig.facecolor'] = 'white'

rcParams['lines.linewidth'] = 1.5
rcParams['lines.markersize'] = 10

# Box around plot
rcParams['axes.linewidth'] = 0.4
rcParams['axes.edgecolor'] = 'black'

# Add ticks
rcParams['xtick.major.size'] = 5
rcParams['ytick.major.size'] = 5

rcParams['legend.fontsize'] = 12
rcParams['legend.edgecolor'] = 'black'

In [ ]:
# Add an extra color to the default cycle to avoid reusing colors
current_colors = rcParams['axes.prop_cycle'].by_key()['color']

# Add a new color
new_color = '#6a0dad'  # Example color, replace with the color you want to add
updated_colors = current_colors + [new_color]

# Set the updated color cycle
plt.rc('axes', prop_cycle=(plt.cycler('color', updated_colors)))

In [ ]:
# Only for seeing what the plots look like
x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.figure(figsize=(6, 4))
plt.plot(x, y, label='sin(x)')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('Example Plot')
plt.legend()
plt.show()

## Data preparation

In [ ]:
# Load simulation profile
profiles_dir = Path("../torque_profiles")

with open(profiles_dir / "simulation_profile.csv", "r") as file:
    sim_profile = pd.read_csv(file)

In [ ]:
# Only to be used for EMG plots
latex_figure_dir = Path("/Users/xabieririzar/Desktop/Life/Studium/TUM/Master_Robotics/Harvard/Thesis/Thesis LaTeX/figures/")

In [ ]:
# Load simulation profile
profiles_dir = Path("../torque_profiles")

with open(profiles_dir / "simulation_profile.csv", "r") as file:
    sim_profile = pd.read_csv(file)

### Load Data

In [ ]:
target_splines = []

for file in sorted((profiles_dir).iterdir()):
    if '62' in file.name:
        target_splines.append(pd.read_csv(file))

In [ ]:
subject_dir = Path(f"../subject_logs/")
subjects = sorted([subject for subject in subject_dir.iterdir() if subject.is_dir()])

subject_data = {}

subject_dirs = {}

for subject in subjects:
    subject_name = subject.name
    subject_data[subject_name] = {}
    subject_dirs[subject_name] = {}

    for session in subject.iterdir():
        if session.is_dir():
            session_dir = session
            subject_data[subject_name][session.name] = {}
            subject_dirs[subject_name][session.name] = {}

            with open("../motor_config.yaml", "r") as f:
                motor_config = yaml.load(f, Loader=yaml.FullLoader)

            angle_calibration_path = session_dir / "device_height_calibration.yaml"

            emg_dir = session_dir / "EMG"
            plot_dir = session_dir / "plots"
            plot_dir.mkdir(exist_ok=True)

            subject_dirs[subject_name][session.name]["emg_dir"] = emg_dir
            subject_dirs[subject_name][session.name]["plot_dir"] = plot_dir
            subject_dirs[subject_name][session.name]["angle_calibration_path"] = angle_calibration_path

            # Read EMG file to extract configuration
            emg_file = emg_dir / "assist_0001_emg.tsv" # 2 on treadmill room, 3 on track room
            emg_config = dict()

            relevant_headers = ["FREQUENCY", "CHANNEL_NAMES"]

            for header in read_headers(emg_file, 13, delimiter="\t"):
                if header[0] in relevant_headers:
                    if header[0] == "FREQUENCY":
                        emg_config[header[0]] = float(header[1])
                    else:
                        emg_config[header[0]] = header[1:]
            print("EMG_CONFIG Channel Names:", emg_config["CHANNEL_NAMES"], "\n")
            subject_data[subject_name][session.name]["emg_config"] = emg_config

In [ ]:
mapping = {}
for channel in emg_config["CHANNEL_NAMES"]:
    relevant_part = channel.split(" ")[3:]
    side = relevant_part[0].split("_")[1].upper()

    muscle = "".join([word[0] for word in relevant_part[1:]])

    mapping[channel] = "_".join([muscle, side])
mapping

In [ ]:
for subject in subject_data:
    for session in subject_data[subject]:
        if subject == "MIH01":
            mapping = {
            "Trigno Avanti Duo Mini1_EMG 1": "RF_RIGHT", 
            "Trigno Avanti Duo Mini1_EMG 2": "VM_RIGHT", 
            "Trigno Avanti Duo Mini2_EMG 1": "RF_LEFT",
            "Trigno Avanti Duo Mini2_EMG 2": "VM_LEFT",
        }
        elif subject == "MIH02":
            # mapping = {
            #     "Trigno Avanti Duo Mini1_EMG BLUE": "VM_RIGHT",
            #     "Trigno Avanti Duo Mini1_EMG YELLOW": "RF_RIGHT",
            #     "Trigno Avanti Duo Mini2_EMG BLUE": "VM_LEFT",
            #     "Trigno Avanti Duo Mini2_EMG YELLOW": "RF_LEFT",
            #     "Trigno Avanti Duo Mini3_EMG BLUE": "BF_RIGHT_1",
            #     "Trigno Avanti Duo Mini3_EMG YELLOW": "BF_RIGHT_2",
            #     "Trigno Avanti Duo Mini4_EMG BLUE": "BF_LEFT_1",
            #     "Trigno Avanti Duo Mini4_EMG YELLOW": "BF_LEFT_2",
            # }
            mapping = {
                'Trigno Avanti Duo Mini1_Right Vastus Medialis': 'VM_RIGHT',
                'Trigno Avanti Duo Mini1_Right Rectus Femoris': 'RF_RIGHT',
                'Trigno Avanti Duo Mini2_Left Vastus Medialis': 'VM_LEFT',
                'Trigno Avanti Duo Mini2_Left Rectus Femoris': 'RF_LEFT',
                'Trigno Avanti Duo Mini3_Right Biceps Femoris 1': 'BF_RIGHT_1',
                'Trigno Avanti Duo Mini3_Right Biceps Femoris 2': 'BF_RIGHT_2',
                'Trigno Avanti Duo Mini4_Left Biceps Femoris 1': 'BF_LEFT_1',
                'Trigno Avanti Duo Mini4_Left Biceps Femoris 2': 'BF_LEFT_2'
            }
        elif subject == "XABI":
            continue
            mapping = {
                "Trigno Avanti Duo Mini1_EMG 2": "RF_RIGHT",
                "Trigno Avanti Duo Mini2_EMG 1": "RF_LEFT"
            }
        session_dir = subject_dir / subject / session

        subject_data[subject][session]["emg_config"]["MAPPING"] = mapping

        if not (session_dir / "emg_config.yaml").exists():
            print("Writing EMG config...")
            with open(session_dir / "emg_config.yaml", "w") as f:
                yaml.dump(emg_config, f)
        else:
            print("EMG config already exists")
            # Open existing config
            with open(session_dir / "emg_config.yaml", "r") as f:
                emg_config = yaml.load(f, Loader=yaml.FullLoader)

        with open(subject_dirs[subject][session]["angle_calibration_path"] , 'r') as f:
            angle_calibration = yaml.load(f, Loader=yaml.FullLoader)

        subject_data[subject][session]["angle_calibration"] = angle_calibration

In [ ]:
session_dict = {}
session_dict["MVIC"] = {}
session_dict["MVIC"]["LEFT"] = None
session_dict["MVIC"]["RIGHT"] = None

data_dict = {"EMG": [],
             "MOTOR_DATA": [],
             "MARKER_DATA": []}

session_dict["UNPOWERED"] = {}
session_dict["ASSISTED"] = {}
session_dict["UNPOWERED"]["BEFORE"] = deepcopy(data_dict)
session_dict["UNPOWERED"]["AFTER"] = deepcopy(data_dict)

# Load all session data
profile_to_num = {}
profile_infos = {}

# Skip first 0.5s to avoid initial noise
skip_first = 0.5 #s
remove_unpowered_after = True

# Handle first motor data
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = deepcopy(session_dict)
        for file_path in sorted(session.iterdir()):
            if file_path.suffix == ".csv":
                if "unpowered" in file_path.stem:
                    df = pd.read_csv(file_path, index_col="time").loc[skip_first:]
                    n = int(file_path.stem.split("_")[1])
                    if n <= 5:
                        session_data["UNPOWERED"]["BEFORE"]["MOTOR_DATA"].append(df)
                    else:
                        remove_unpowered_after = False
                        session_data["UNPOWERED"]["AFTER"]["MOTOR_DATA"].append(df)
                elif "assist" in file_path.stem:
                    df = pd.read_csv(file_path, skiprows=2, index_col="time").loc[skip_first:]

                    profile_info = read_headers(file_path=file_path, rows=2, delimiter=",")
                    time = int(profile_info[0][1])
                    force = int(profile_info[1][1])

                    profile = f"peak_{time}%_{force}N"
                    profile_infos[profile] = {"peak_time": time, "peak_force": force}

                    if profile not in profile_to_num.keys():
                        profile_to_num[profile] = []

                    num = int(file_path.stem.split("_")[-1])
                    profile_to_num[profile].append(num)

                    if profile not in session_data["ASSISTED"].keys():
                        session_data["ASSISTED"][profile] = deepcopy(data_dict)

                    session_data["ASSISTED"][profile]["MOTOR_DATA"].append(df)
        if remove_unpowered_after:
            session_data["UNPOWERED"].pop("AFTER")
        subject_data[subject.name][session.name]["session_data"] = session_data

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        emg_dir = session / "EMG"
        session_data = subject_data[subject.name][session.name]["session_data"]
        emg_config = subject_data[subject.name][session.name]["emg_config"]
        
        relevant_cols = ["TIME"] + list(emg_config["MAPPING"].keys())
        
        # Handle second EMG data
        for file_path in sorted(emg_dir.iterdir()):
            if file_path.suffix == ".tsv":
                if "unpowered" in file_path.stem:
                    n = int(file_path.stem.split("_")[1])

                    if "marker" in file_path.stem:
                        # Convert mm to meters
                        df = pd.read_csv(file_path, sep="\t", skiprows=11, usecols=["Time", "Hip Left Z", "Hip Right Z"], index_col="Time") / 1000
                        df = df.loc[skip_first:]
                        if n <= 5:
                            session_data["UNPOWERED"]["BEFORE"]["MARKER_DATA"].append(df)
                        else:
                            session_data["UNPOWERED"]["AFTER"]["MARKER_DATA"].append(df)
                    else:
                        df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                        df.rename(columns=emg_config["MAPPING"], inplace=True)
                        df.sort_index(axis=1, inplace=True)
                        df = df.loc[skip_first:]

                        if n <= 5:
                            session_data["UNPOWERED"]["BEFORE"]["EMG"].append(df)
                        else:
                            session_data["UNPOWERED"]["AFTER"]["EMG"].append(df)

                elif "assist" in file_path.stem:
                    if "marker" in file_path.stem:
                        df = pd.read_csv(file_path, sep="\t", skiprows=11, usecols=["Time", "Hip Left Z", "Hip Right Z"], index_col="Time") / 1000
                        df = df.loc[skip_first:]
                        data_type = "MARKER_DATA"
                    else:
                        df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                        df.rename(columns=emg_config["MAPPING"], inplace=True)
                        df.sort_index(axis=1, inplace=True)
                        df = df.loc[skip_first:]
                        data_type = "EMG"
                        
                    num = int(file_path.stem.split("_")[1])

                    for profile, nums in profile_to_num.items():
                        if num in nums:
                            session_data["ASSISTED"][profile][data_type].append(df)
                            
                elif "MVIC" in file_path.stem:
                    df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                    df.rename(columns=emg_config["MAPPING"], inplace=True)
                    
                    side = "LEFT" if "left" in file_path.stem.lower() else "RIGHT"
                    side_prefix = "_" + side

                    if "BF" in file_path.name:  # If it's the BF file
                        bf_col = [col for col in df.columns if "BF" + side_prefix in col]  # Adjust this if your naming convention is different
                        df = df[bf_col]
                    
                    elif "RF" in file_path.name or "VM" in file_path.name:  # If it's the RF or VM file
                        rf_vm_cols = [col for col in df.columns if any(sub in col for sub in ["RF" + side_prefix, "VM" + side_prefix])]  # Adjust this if your naming convention is different
                        df = df[rf_vm_cols]
                    
                    if session_data["MVIC"][side] is not None:
                        # Merge the new data with the existing data
                        session_data["MVIC"][side] = pd.merge(session_data["MVIC"][side], df, left_index=True, right_index=True, how='outer')
                    else:
                        # Initialize with the new data
                        session_data["MVIC"][side] = df

In [ ]:
# Get profile names for plots
profile_names_paper = {profile: f"{profile.split('_')[1]} peak timing" for profile in profile_to_num.keys()}

### Drop unnecessary muscles

The idea here is that the recording from some EMG sensors can be noisy, hence we choose the one that shows consistent results with literature.

IMPORTANT: This can only be done after analyzing the data, so make sure to first look at MVIC and corresponding muscle activations

In [ ]:
def get_unique_muscles(session_data):
    unique_muscles = [
        re.sub(pattern=r"(_LEFT|LEFT_|_RIGHT|RIGHT_)", repl="", string=col)  # Remove LEFT/RIGHT variations
        for col in session_data["MVIC"]["LEFT"].columns
    ]
    return unique_muscles

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        unique_muscles = get_unique_muscles(session_data)
        print(f"Subject: {subject.name}, Session: {session.name}, Unique Muscles: {unique_muscles}")

In [ ]:
session_data = subject_data["MIH02"]["December_20"]["session_data"]

to_drop = ["BF_LEFT_2", "BF_RIGHT_1"] # For MIH02

for side in session_data["MVIC"].keys():
    emg_df = session_data["MVIC"][side]
    emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
    col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}

    emg_df.rename(columns=col_mapping, inplace=True)

for t_unpowered in session_data["UNPOWERED"].keys():
    for emg_df in session_data["UNPOWERED"][t_unpowered]["EMG"]:
        emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
        col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}
        emg_df.rename(columns=col_mapping, inplace=True)

for profile in session_data["ASSISTED"].keys():
    for data_type in session_data["ASSISTED"][profile].keys():
        if data_type != "EMG":
            continue
        for emg_df in session_data["ASSISTED"][profile][data_type]:
            emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
            col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}
            emg_df.rename(columns=col_mapping, inplace=True)

unique_muscles = get_unique_muscles(session_data)

### Remove incomplete data (if any)

Sometimes motor data collection may go wrong and result in empty dataframes or durations that are significantly shorter in time than EMG or MARKER data

In [ ]:
# Track which dataframe is removed for being empty (profile, datatype and index) and for what reason
removed_dfs = {}

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        for profile in session_data["ASSISTED"].keys():
            non_empty_dfs = [(emg_df, motor_df, marker_df) for emg_df, motor_df, marker_df in zip(*session_data["ASSISTED"][profile].values()) if not emg_df.empty and not motor_df.empty and not marker_df.empty]
            for i, (emg_df, motor_df, marker_df) in enumerate(zip(*session_data["ASSISTED"][profile].values())):
                emg_df.dropna(inplace=True)
                motor_df.dropna(inplace=True)
                marker_df.dropna(inplace=True)
                
                if emg_df.empty:
                    removed_dfs[(profile, "EMG", i)] = "empty"
                    session_data["ASSISTED"][profile]["EMG"][i] = non_empty_dfs[0][0].copy()
                    session_data["ASSISTED"][profile]["EMG"][i].loc[:] = 1

                if motor_df.empty:
                    removed_dfs[(profile, "MOTOR_DATA", i)] = "empty"
                    session_data["ASSISTED"][profile]["MOTOR_DATA"][i] = non_empty_dfs[0][1].copy()
                    session_data["ASSISTED"][profile]["MOTOR_DATA"][i].loc[:] = 1

                if marker_df.empty:
                    removed_dfs[(profile, "MARKER_DATA", i)] = "empty"
                    session_data["ASSISTED"][profile]["MARKER_DATA"][i] = non_empty_dfs[0][2].copy()
                    session_data["ASSISTED"][profile]["MARKER_DATA"][i].loc[:] = 1
                    
        print(subject / session)
        print(removed_dfs)

## Filter EMG data

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        subject_data[subject.name][session.name]["filtered_session_data"] = deepcopy(session_data)
        
        filtered_session_data = subject_data[subject.name][session.name]["filtered_session_data"]

        filtered_session_data["MVIC"]["LEFT"], _ = filter_emg(session_data["MVIC"]["LEFT"], sfreq=emg_config["FREQUENCY"])
        filtered_session_data["MVIC"]["RIGHT"], _ = filter_emg(session_data["MVIC"]["RIGHT"], sfreq=emg_config["FREQUENCY"])

        for profile, emg_data in session_data["ASSISTED"].items():
            for i, emg_df in enumerate(session_data["ASSISTED"][profile]["EMG"]):
                filtered_session_data["ASSISTED"][profile]["EMG"][i], _ = filter_emg(emg_df, sfreq=emg_config["FREQUENCY"])

        for unpow_key in session_data["UNPOWERED"].keys():
            for i, emg_df in enumerate(session_data["UNPOWERED"][unpow_key]["EMG"]):
                filtered_session_data["UNPOWERED"][unpow_key]["EMG"][i], _ = filter_emg(emg_df, sfreq=emg_config["FREQUENCY"])

### Plot MVIC data

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():    
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        n_cols = filtered_session_data["MVIC"]["LEFT"].shape[1]

        fig, axs = plt.subplots(1, n_cols, figsize=(3*n_cols, 3), sharey=True)
        fig.suptitle(f"Subject {subject} MVIC")

        for i, side in enumerate(filtered_session_data["MVIC"].keys()):
            for j, col in enumerate(filtered_session_data["MVIC"][side].columns):
                if n_cols == 1:
                    axs.plot(filtered_session_data["MVIC"][side].index, filtered_session_data["MVIC"][side][col], label=side)
                    axs.set_title(f"{col}")
                    axs.set_xlabel("Time (s)")
                    axs.set_ylabel("EMG signal (mV)")
                    handles, labels = axs.get_legend_handles_labels()
                else:
                    axs[j].plot(filtered_session_data["MVIC"][side].index, filtered_session_data["MVIC"][side][col], label=side)
                    axs[j].set_title(f"{col.split('_')[0]}")
                    axs[j].set_xlabel("Time (s)")
                    axs[0].set_ylabel("EMG signal (mV)")
                    handles, labels = axs[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right', fontsize=10, bbox_to_anchor=(0.85, 1))

        plt.tight_layout()
        plt.savefig(subject_dirs[subject][session]['plot_dir'] / f"{subject}_MVIC.svg", dpi=500, bbox_inches='tight', format='svg')
        plt.savefig(latex_figure_dir / "emg_plots" / f"{subject}_MVIC.png", dpi=500, format="png")

### Plot UNFILTERED vs FILTERED EMG

### Plot every muscle activations across iterations and profiles

In [ ]:
session_data["ASSISTED"]['peak_27%_62N']['EMG'][0][["VM_LEFT", "RF_LEFT"]].plot()
filtered_session_data["ASSISTED"]['peak_27%_62N']['EMG'][0][["VM_LEFT", "RF_LEFT"]].plot()

In [ ]:
subject = "MIH01"
session = "December_19"

session_data = subject_data[subject][session]["session_data"]
filtered_session_data = subject_data[subject][session]["filtered_session_data"]

unique_muscles = get_unique_muscles(session_data)

for muscle in unique_muscles:
    for profile in session_data["ASSISTED"].keys():
        plot_dir = subject_dirs[subject][session]["plot_dir"]
        filename = plot_dir / f"EMG_{muscle}_{profile}.svg"

        plot_muscle_emg(title=f"{muscle} EMG data (unscaled)\n{profile_names_paper[profile]}", 
                target_muscle=muscle,
                unfiltered_dfs=session_data["ASSISTED"][profile]["EMG"],
                filtered_dfs=filtered_session_data["ASSISTED"][profile]["EMG"], 
                freq=emg_config["FREQUENCY"],
                fig_path=filename,
                show=False)

### Plot UNFILTERED vs FILTERED EMG

### Plot every muscle activations across iterations and profiles`

In [ ]:
subject = "MIH02"
session = "December_20"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
session_data = subject_data[subject][session]["session_data"]

unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
for muscle in unique_muscles:
    for timing in session_data["UNPOWERED"].keys():
        plot_dir = subject_dirs[subject][session]["plot_dir"]
        filename = plot_dir / f"EMG_unpowered_{muscle}_{timing}.png"
        plot_muscle_emg(title=f"Subject {subject} EMG data (unscaled)\n{timing}", 
                target_muscle=muscle,
                unfiltered_dfs=session_data["UNPOWERED"][timing]["EMG"],
                filtered_dfs=filtered_session_data["UNPOWERED"][timing]["EMG"],
                fig_path=filename,
                show=False)






### Scale EMG data

#### Scale based on MVIC

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        left_mvic_cols = list(filtered_session_data["MVIC"]["LEFT"].columns)
        right_mvic_cols = list(filtered_session_data["MVIC"]["RIGHT"].columns)

        max_left = list(filtered_session_data["MVIC"]["LEFT"].max())
        max_right = list(filtered_session_data["MVIC"]["RIGHT"].max())

        # Create dataframe containing the max values for each muscle, should be 1 row
        mvic_df = pd.DataFrame(data=[max_left + max_right], columns=left_mvic_cols + right_mvic_cols)
        mvic_df = mvic_df[filtered_session_data["UNPOWERED"]["BEFORE"]["EMG"][0].columns]
        
        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            for i, unpowered_df in enumerate(filtered_session_data["UNPOWERED"][t_unpowered]["EMG"]):
                unpowered_df /= mvic_df.iloc[0]

        for profile in filtered_session_data["ASSISTED"].keys():
            for i, filtered_emg in enumerate(filtered_session_data["ASSISTED"][profile]["EMG"]):
                filtered_emg /= mvic_df.iloc[0]

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
session_data = subject_data[subject][session]["session_data"]

unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
for muscle in unique_muscles:
    for profile in session_data["ASSISTED"].keys():
        plot_dir = subject_dirs[subject][session]["plot_dir"]
        filename = plot_dir / f"EMG_scaled_{muscle}_{profile}.png"
        plot_muscle_emg(title=f"Subject {subject} EMG data (scaled)\n{profile_names_paper[profile]}", 
                target_muscle=muscle,
                unfiltered_dfs=session_data["ASSISTED"][profile]["EMG"],
                filtered_dfs=filtered_session_data["ASSISTED"][profile]["EMG"],
                fig_path=filename,
                show=False)

#### Scale EMG based on reported peak activation (based on papers)

### Plot UNFILTERED vs FILTERED EMG

In [ ]:
enable_paper_scaling = False

# Based on target muscles
muscle_values = {
    "RF": 1,#0.46,
    "BF":1,# 0.16,
    "VM": 0.9
}

activation_scaling_factor = {
    "RF_RIGHT": 1,
    "RF_LEFT": 1,
    "BF_RIGHT": 1,
    "BF_LEFT": 1,
    "VM_RIGHT": 1,
    "VM_LEFT": 1
}
subject = "MIH02"
session = "December_20"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

if enable_paper_scaling:
    for emg_data in filtered_session_data["UNPOWERED"]["BEFORE"]["EMG"]:
        for muscle in emg_data.columns:
            # if "RF" in muscle:
            #     activation_scaling_factor[muscle] = emg_data[muscle].max() / muscle_values["RF"]
            #     emg_data[muscle] /= activation_scaling_factor[muscle] 
            # elif "BF" in muscle:
            #     activation_scaling_factor[muscle] = emg_data[muscle].max() / muscle_values["BF"]
            #     emg_data[muscle] /= activation_scaling_factor[muscle] 
            if "VM" in muscle:
                activation_scaling_factor[muscle] = emg_data[muscle].max() / muscle_values["VM"]
                emg_data[muscle] /= activation_scaling_factor[muscle] 

    for profile in filtered_session_data["ASSISTED"].keys():
        for emg_data in filtered_session_data["ASSISTED"][profile]["EMG"]:
            for muscle in emg_data.columns:
                emg_data[muscle] /= activation_scaling_factor[muscle]

### Plot SCALED and FILTERED EMG data

In [ ]:
subject = "MIH01"
session = "December_19"
profile = "peak_47%_62N"

unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"77_peak_every_muscle_activation.svg"

plot_every_muscle(title=f"Assisted EMG activations (scaled)\n{profile_names_paper[profile]}", muscles=unique_muscles, dfs=subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["EMG"],
                  fig_path=filename)

In [ ]:
subject = "MIH02"
session = "December_20"

t_unpowered = "BEFORE"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"unassisted_every_muscle_before.svg"

plot_every_muscle(title=f"Unassisted EMG activations (scaled)\n{t_unpowered}", muscles=unique_muscles, dfs=filtered_session_data["UNPOWERED"][t_unpowered]["EMG"][:5],
                  fig_path=filename)

## Process MOTOR data

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

motor_log = subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["MOTOR_DATA"][0]

colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"motor_unfiltered.svg"

fig, ax = plt.subplots(3, 1, sharex=True, figsize=(8, 5))
fig.suptitle(profile_names_paper[profile])

ax[0].plot(motor_log.index, motor_log["theta_2"], label=r"$\theta_2$", color=colors.__next__())
# ax[0].axhline(y=angle_calibration["new_range"]["min"], linestyle="--", color="black")
# ax[0].axhline(y=angle_calibration["new_range"]["max"], linestyle="--", color="black")
handles0, labels0 = ax[0].get_legend_handles_labels()
ax[0].set_ylabel(r"$\theta_2$ (rad)")


ax[1].plot(motor_log.index, motor_log["target_tau_1"], label=r"Target $\tau_1$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_1"], label=r"Measured $\tau_1$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["target_tau_2"], label=r"Target $\tau_2$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_2"], label=r"Measured $\tau_2$", color=colors.__next__())
handles1, labels1 = ax[1].get_legend_handles_labels()
ax[1].set_ylabel("Torques (Nm)", fontsize=12)

ax[2].plot(motor_log.index, motor_log["Percentage"], label="STS %", color=colors.__next__())
ax[2].axhline(y=100, linestyle="--", color="black")
ax[2].axhline(y=0, linestyle="--", color="black")
handles2, labels2 = ax[2].get_legend_handles_labels()
ax[2].set_ylabel("STS %")
ax[2].set_xlabel('time(s)')

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2
fig.legend(handles, labels, loc='upper center', ncols=6, bbox_to_anchor=(0.5, 0.93), fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.show()

### Fix wrap-around from motor measurements

In [ ]:
# Control variable to toggle the sign
torque_limits = [motor_config[motor]["T_max"] for motor in motor_config.keys()]

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        for profile in filtered_session_data["ASSISTED"].keys():
            motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]

            measured_torques = [column for column in motor_data[0].columns if "measured_tau" in column]

            for motor_df in motor_data:
                for torque, jump_threshold in zip(measured_torques, torque_limits):
                    differences = motor_df[torque].diff().fillna(0)

                    # Threshold for detecting a jump
                    toggle_sign = 1

                    corrected_values = []

                    for diff in differences:
                        if abs(diff) > jump_threshold:
                            toggle_sign *= -1
                        corrected_values.append(toggle_sign)
                    motor_df[torque] *= corrected_values

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

motor_log = subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["MOTOR_DATA"][0]

colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"motor_filtered.svg"


fig, ax = plt.subplots(3, 1, sharex=True, figsize=(8, 5))
fig.suptitle(profile_names_paper[profile])

ax[0].plot(motor_log.index, motor_log["theta_2"], label=r"$\theta_2$", color=colors.__next__())
handles0, labels0 = ax[0].get_legend_handles_labels()
ax[0].set_ylabel(r"$\theta_2$ (rad)")


ax[1].plot(motor_log.index, motor_log["target_tau_1"], label=r"Target $\tau_1$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_1"], label=r"Measured $\tau_1$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["target_tau_2"], label=r"Target $\tau_2$", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_2"], label=r"Measured $\tau_2$", color=colors.__next__())
handles1, labels1 = ax[1].get_legend_handles_labels()
ax[1].set_ylabel("Torques (Nm)", fontsize=12)

ax[2].plot(motor_log.index, motor_log["Percentage"], label="STS %", color=colors.__next__())
ax[2].axhline(y=100, linestyle="--", color="black")
ax[2].axhline(y=0, linestyle="--", color="black")
handles2, labels2 = ax[2].get_legend_handles_labels()
ax[2].set_ylabel("STS %")
ax[2].set_xlabel('time(s)')

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2
fig.legend(handles, labels, loc='upper center', ncols=6, bbox_to_anchor=(0.5, 0.93), fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.show()

### Calculate force from torques

In [ ]:
rotate_90 = get_rotation_matrix(90)

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
                
        for profile in filtered_session_data["ASSISTED"].keys():
            for motor_log in filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]:
                measured_torques = motor_log[["measured_tau_1", "measured_tau_2"]].to_numpy()
                measured_torques = measured_torques.reshape(*measured_torques.shape, 1)

                jacobians = get_jacobian(l1=0.44, l2=0.41, theta_1=motor_log["theta_1"].to_numpy(), theta_2=motor_log["theta_2"].to_numpy())
                F = - np.linalg.inv(jacobians.T) @ measured_torques
                # F = - rotate_90[:2, :2] @ np.linalg.inv(jacobians.T) @ measured_torques

                motor_log["F_X"] = F[:, 0]
                motor_log["F_Y"] = F[:, 1]
                

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"overview_force_muscle.svg"

n_cols = len(filtered_session_data["ASSISTED"].keys())
n_rows = len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"])

target_muscle = "BF_RIGHT"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 10), sharey=True)
fig.suptitle(f"Subject {subject} EMG and force data")

for i, profile in enumerate(filtered_session_data["ASSISTED"].keys()):
    axs[0, i].set_title(f"{profile_names_paper[profile]}")
    for j, motor_df in enumerate(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]):
        emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][j]

        # Plot force values on the primary y-axis
        axs[j, i].plot(motor_df.index, motor_df.F_Y, label=r"Applied force $F_Y$", color='orange')
        axs[j, i].tick_params(axis='y', labelcolor='orange')

        # Create a secondary y-axis for EMG data
        ax2 = axs[j, i].twinx()
        ax2.plot(emg_data.index, emg_data[target_muscle], label=target_muscle, color=u'#1f77b4')
        ax2.set_ylim(0, 1)  # Set EMG activation scale from 0 to 1
        
        if i == len(filtered_session_data["ASSISTED"].keys()) - 1:
            ax2.set_ylabel('EMG (% MVIC)', color='black')
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelcolor=u'#1f77b4')

        axs[j, 0].set_ylabel(f"Iteration {j+1}\nForce (N)")
        axs[-1, i].set_xlabel("Time (s)")

        handles, labels = axs[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles += handles2
        labels += labels2

fig.legend(handles, labels, loc='upper center', ncols=2, bbox_to_anchor=(0.5, 0.96), fontsize=14)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Remove faulty data (based on missing MOTOR values)

#### Overview of what motor data looks like

In [ ]:
profile = "peak_77%_62N"
iteration = 1

motor_df = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][iteration-1]

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"motor_data_overview.svg"

colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

fig, ax = plt.subplots(1, figsize=(10, 5))
fig.suptitle(f"Motor Data Overview\n{subject} - {profile_names_paper[profile]} - {iteration}")

ax.plot(motor_df.index, motor_df.F_Y, label=r"$F_Y$", color=colors.__next__())

ax2.plot(motor_df.index, motor_df.target_tau_1, label=r"Target $\tau_1$", color=colors.__next__())
ax2 = ax.twinx()
ax2.plot(motor_df.index, motor_df.measured_tau_1, label=r"Measured $\tau_1$", color=colors.__next__())
ax2.plot(motor_df.index, motor_df.target_tau_2, label=r"Target $\tau_2$", color=colors.__next__())
ax2.plot(motor_df.index, motor_df.measured_tau_2, label=r"Measured $\tau_2$", color=colors.__next__())
ax2.set_ylabel("Torque (Nm)")


ax.set_ylabel("Applied Force (N)")
ax.set_xlabel("STS (%)")

handles1, labels1 = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

handles = handles1 + handles2
labels = labels1 + labels2

fig.legend(handles, labels, ncols=len(handles), bbox_to_anchor=[0.8, 0.9])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

## Extract relevant STS time window based on HIP height

In [ ]:
time_range = namedtuple("TimeRange", ["min", "max"])

### Calculate relevant time window

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]              

        relevant_time_ranges = {}

        for profile in filtered_session_data["ASSISTED"].keys():
            # relevant_time_ranges[profile] = []
            relevant_time_ranges[profile] = {"ranges": [], "hip_vel": []}

            for marker_data in filtered_session_data["ASSISTED"][profile]["MARKER_DATA"]:
                hip_vel = smooth_dataframe(marker_data["Hip Left Z"].diff() / marker_data.index.diff(), window_size=30)

                baseline = hip_vel.loc[:1]
                baseline_avg = baseline.mean() + 0.05 # add offset to be safe

                condition = hip_vel > baseline_avg

                first_index = condition.idxmax() - 0.2
                last_index = condition.iloc[::-1].idxmax() + 0.7

                relevant_time_ranges[profile]["ranges"].append(time_range(first_index, last_index))
                relevant_time_ranges[profile]["hip_vel"].append(hip_vel)

        for unpowered_time in filtered_session_data["UNPOWERED"].keys():
            relevant_time_ranges[f"UNPOWERED_{unpowered_time}"] = {"ranges": [], "hip_vel": []}

            for marker_data in filtered_session_data["UNPOWERED"][unpowered_time]["MARKER_DATA"]:
                hip_vel = smooth_dataframe(marker_data["Hip Left Z"].diff() / marker_data.index.diff(), window_size=30)

                baseline = hip_vel.loc[:1]
                baseline_avg = baseline.mean() + 0.05

                condition = hip_vel > baseline_avg

                first_index = condition.idxmax() - 0.2
                last_index = condition.iloc[::-1].idxmax() + 0.7

                relevant_time_ranges[f"UNPOWERED_{unpowered_time}"]["ranges"].append(time_range(first_index, last_index))
                relevant_time_ranges[f"UNPOWERED_{unpowered_time}"]["hip_vel"].append(hip_vel)
        
        subject_data[subject][session]["relevant_time_ranges"] = relevant_time_ranges

### Average STS duration across profiles and subjects

In [ ]:
# Extract durations from the time ranges
fig, ax = plt.subplots(1, ncols=len(subject_data.keys()), figsize=(10, 5))
fig.suptitle("Durations per profile and subject")

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

        durations = {profile: [time_range.max - time_range.min for time_range in time_ranges["ranges"]] for profile, time_ranges in relevant_time_ranges.items()}

        
        # Drop empty keys
        durations = {k: v for k, v in durations.items() if v}

        for old_key, new_key in profile_names_paper.items():
            durations[new_key] = durations.pop(old_key)
        durations = dict(sorted(durations.items()))

        durations_df = pd.DataFrame.from_dict(durations, orient='index').T

        # Plot mean and std of the durations
        durations_df.mean().plot(kind='bar', yerr=durations_df.std(), ax=ax[i])
        ax[i].set_ylabel("Duration (s)")
        ax[i].set_xlabel("Profile")
        ax[i].set_title(f"Subject {subject}")
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right')
        plt.tight_layout()
plt.savefig("../paper_figures/durations_per_profile.svg", format='svg')
plt.savefig("../paper_figures/durations_per_profile.png", dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / "durations_per_profile.png", dpi=500, format="png")
plt.show()


### Time windows across all iterations

In [ ]:
for profile in relevant_time_ranges.keys():
    print(profile, relevant_time_ranges[profile]["ranges"])

### Time windows across all iterations

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

profiles = filtered_session_data["ASSISTED"].keys()
n_rows = len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"extracting_time_window.svg"

fig, axs = plt.subplots(n_rows, len(profiles), figsize=(n_rows * 3, 1.5 * n_rows), sharey=True)  # Adjust as needed
fig.suptitle(f"Extracting time window based on speed")

for col, profile in enumerate(profiles):
    for row in range(n_rows):
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

        marker_df = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][row]  # Adjusted for demonstration
        cur_range = relevant_time_ranges[profile]["ranges"][row]  # Assuming relevant_time_ranges is structured similarly
        hip_vel = relevant_time_ranges[profile]["hip_vel"][row]  # Assuming relevant_time_ranges is structured similarly

        axs[row, col].plot(marker_df.index, marker_df["Hip Left Z"], label="Height", color=u'#1f77b4')
        axs[row, col].tick_params(axis='y', labelcolor=u'#1f77b4')
        axs[row, col].axvline(cur_range.min, color="red", linestyle="--")
        axs[row, col].axvline(cur_range.max, color="red", linestyle="--")

        ax2 = axs[row, col].twinx()
        ax2.plot(hip_vel.index, hip_vel, label="Speed", color="orange")
        ax2.tick_params(axis='y', labelcolor="orange")
        if col == len(profiles) - 1:
            ax2.set_ylabel("Speed (m/s)")

        handles1, labels1 = axs[row, col].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2


        axs[0, col].set_title(profile_names_paper[profile])
        axs[row, 0].set_ylabel(f"Iteration {row+1}\nHeight (m)")
        axs[-1, col].set_xlabel("Time (s)")

        
fig.legend(handles, labels, loc="upper center", ncols=len(handles), bbox_to_anchor=(0.5, 0.95))
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

filename = subject_dirs[subject][session]["plot_dir"] / f"unpowered_activations.svg"

fig, axs = plt.subplots(nrows=3, ncols=5, sharex='col', figsize=(20, 5))
fig.suptitle(f"Unpowered activations (BEFORE trials)\n{subject} - {profile_names_paper[profile]}")

t_unpowered = "BEFORE"


target_muscle = random.choice(unique_muscles)

for i, t_range in enumerate(relevant_time_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"]):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])
    marker_data = filtered_session_data["UNPOWERED"][t_unpowered]["MARKER_DATA"][i] 
    motor_data = filtered_session_data["UNPOWERED"][t_unpowered]["MOTOR_DATA"][i]
    emg_data = filtered_session_data["UNPOWERED"][t_unpowered]["EMG"][i]

    axs[0, i].set_title(f"unpowered_{i}")
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"], label="Hip Z", color=colors.__next__())
    axs[0, i].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(t_range.min, linestyle='--', color="red")
    axs[0, i].axvline(t_range.max, linestyle='--', color="red")
    
    handles0, labels0 = axs[0, i].get_legend_handles_labels()

    axs[1, i].plot(motor_data.index, motor_data["theta_2"], colors.__next__(), label=r"$\theta_2$")
    axs[1, i].set_ylabel(r"$\theta_2$ (rad)")
    axs[1, i].axvline(t_range.min, linestyle='--', color="red")
    axs[1, i].axvline(t_range.max, linestyle='--', color="red")

    handles1, labels1 = axs[1, i].get_legend_handles_labels()

    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle}_LEFT", color=colors.__next__())
    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle}_RIGHT", color=colors.__next__())
    axs[2, i].set_xlabel("Time (s)")
    axs[2, i].set_ylabel("EMG (% of MVIC)")
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[2, i].axvline(t_range.min, linestyle='--', color="red")
    axs[2, i].axvline(t_range.max, linestyle='--', color="red")

    axs[2, i].set_ylim(0, 1)

    handles2, labels2 = axs[2, i].get_legend_handles_labels()

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2

fig.legend(handles, labels, loc='upper center', ncols=len(handles), bbox_to_anchor=(0.5, 0.9), fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.show()

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_37%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

filename = subject_dirs[subject][session]["plot_dir"] / f"assisted_activations.svg"

fig, axs = plt.subplots(nrows=3, ncols=5, sharex='col', figsize=(20, 5))
fig.suptitle(f"Assisted activations\n{subject} - {profile_names_paper[profile]}")

target_muscle = random.choice(unique_muscles)

elbow_range = subject_data[subject][session]['angle_calibration']['new_range']

for i, t_range in enumerate(relevant_time_ranges[profile]["ranges"]):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])
    marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] 
    motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i]
    emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][i]

    axs[0, i].set_title(f"Assisted {i}")
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"], label="Hip Z", color=colors.__next__())
    axs[0, i].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(t_range.min, linestyle='--', color="red")
    axs[0, i].axvline(t_range.max, linestyle='--', color="red")
    
    handles0, labels0 = axs[0, i].get_legend_handles_labels()

    axs[1, i].plot(motor_data.index, motor_data["theta_2"], colors.__next__(), label=r"$\theta_2$")
    axs[1, i].set_ylabel(r"$\theta_2$ (rad)")
    axs[1, i].axvline(t_range.min, linestyle='--', color="red")
    axs[1, i].axvline(t_range.max, linestyle='--', color="red")
    axs[1, i].axhline(elbow_range["min"], linestyle='--', color="black")
    axs[1, i].axhline(elbow_range['max'], linestyle='--', color="black")

    axs[1, i].set_ylim(elbow_range["min"] - 0.1, elbow_range["max"] + 0.3)

    handles1, labels1 = axs[1, i].get_legend_handles_labels()

    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle}_LEFT", color=colors.__next__())
    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle}_RIGHT", color=colors.__next__())
    axs[2, i].set_xlabel("Time (s)")
    axs[2, i].set_ylabel("EMG (% of MVIC)")
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[2, i].axvline(t_range.min, linestyle='--', color="red")
    axs[2, i].axvline(t_range.max, linestyle='--', color="red")

    axs[2, i].set_ylim(0, 1)

    handles2, labels2 = axs[2, i].get_legend_handles_labels()

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2

fig.legend(handles, labels, loc='upper center', ncols=len(handles), bbox_to_anchor=(0.5, 0.9), fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Shift motor data that is delayed by more than 0.5s

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_37%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

filename = subject_dirs[subject][session]["plot_dir"] / f"assisted_activations.svg"

fig, axs = plt.subplots(nrows=3, ncols=5, sharex='col', figsize=(20, 5))
fig.suptitle(f"Assisted activations\n{subject} - {profile_names_paper[profile]}")

target_muscle = random.choice(unique_muscles)

elbow_range = subject_data[subject][session]['angle_calibration']['new_range']

for i, t_range in enumerate(relevant_time_ranges[profile]["ranges"]):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])
    marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] 
    motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i]
    emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][i]

    axs[0, i].set_title(f"Assisted {i}")
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"], label="Hip Z", color=colors.__next__())
    axs[0, i].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(t_range.min, linestyle='--', color="red")
    axs[0, i].axvline(t_range.max, linestyle='--', color="red")
    
    handles0, labels0 = axs[0, i].get_legend_handles_labels()

    axs[1, i].plot(motor_data.index, motor_data["theta_2"], colors.__next__(), label=r"$\theta_2$")
    axs[1, i].set_ylabel(r"$\theta_2$ (rad)")
    axs[1, i].axvline(t_range.min, linestyle='--', color="red")
    axs[1, i].axvline(t_range.max, linestyle='--', color="red")
    axs[1, i].axhline(elbow_range["min"], linestyle='--', color="black")
    axs[1, i].axhline(elbow_range['max'], linestyle='--', color="black")

    axs[1, i].set_ylim(elbow_range["min"] - 0.1, elbow_range["max"] + 0.3)

    handles1, labels1 = axs[1, i].get_legend_handles_labels()

    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle}_LEFT", color=colors.__next__())
    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle}_RIGHT", color=colors.__next__())
    axs[2, i].set_xlabel("Time (s)")
    axs[2, i].set_ylabel("EMG (% of MVIC)")
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[2, i].axvline(t_range.min, linestyle='--', color="red")
    axs[2, i].axvline(t_range.max, linestyle='--', color="red")

    axs[2, i].set_ylim(0, 1)

    handles2, labels2 = axs[2, i].get_legend_handles_labels()

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2

fig.legend(handles, labels, loc='upper center', ncols=len(handles), bbox_to_anchor=(0.5, 0.9), fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Shift motor data that is delayed by more than 0.5s

In [ ]:
# In some cases, motor data collection started after EMG, hence we add the time difference to compensate

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        for profile in filtered_session_data["ASSISTED"].keys():
            for i in range(len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])):
                marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] 
                motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i]
                emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][i]

                t_diff = emg_data.index[-1] - motor_data.index[-1]
                if t_diff > 0.5:
                    motor_data.index += t_diff

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_37%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

n_rows = len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"])
n_cols = len(filtered_session_data["ASSISTED"].keys())

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"overview_force_muscle.svg"

target_muscle = "BF"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3, n_rows * 2), sharey='row')
fig.suptitle(f"Subject {subject}, Session {session}\nForce and EMG per profile")

for i, profile in enumerate(filtered_session_data["ASSISTED"].keys()):
    axs[0, i].set_title(profile_names_paper[profile])
    
    for j, motor_df in enumerate(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]):
        emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][j]

        # Plot force values on the primary y-axis
        axs[j, i].plot(motor_df.index, motor_df.F_Y, label=r"$F_Y$", color='orange')
        axs[j, i].tick_params(axis='y', labelcolor='orange')

        # Create a secondary y-axis for EMG data
        ax2 = axs[j, i].twinx()
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label="RIGHT", color='green')
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label="LEFT", color=u'#1f77b4')
        ax2.set_ylim(0, 0.6)  # Set EMG activation scale from 0 to 1

        if i == len(profiles) - 1:
            ax2.set_ylabel('EMG (% MVIC)', color=u'#1f77b4')
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelcolor=u'#1f77b4')

        handles1, labels1 = axs[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2

        axs[j, 0].set_ylabel(f"Iteration {j+1}\nForce (N)")
        axs[-1, i].set_xlabel("Time (s)")

fig.legend(handles, labels, loc="upper right", ncols=len(handles), bbox_to_anchor=(0.6, 0.93))
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
shutil.copy(filename.with_suffix('.png'), latex_figure_dir / "emg_plots")
plt.show()

## Remove iterations with faulty data (both motor or EMG)

### Profile -> iteration mapping to be removed

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        data_to_keep = {key: None for key in filtered_session_data["ASSISTED"].keys()}

        if subject == "MIH01":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [0, 2, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [1, 2, 3]
            data_to_keep['peak_67%_62N'] = [2, 3, 4]
            data_to_keep['peak_77%_62N'] = [1, 2, 3]

        elif subject == "MIH02":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [1, 2, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [0, 2, 3]
            data_to_keep['peak_67%_62N'] = [0, 1, 3]
            data_to_keep['peak_77%_62N'] = [0, 1, 2]

        elif subject == "XABI":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [0, 1, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [0, 2, 3]
            data_to_keep['peak_67%_62N'] = [0, 1, 3]
            data_to_keep['peak_77%_62N'] = [3]
        
        subject_data[subject][session]["data_to_keep"] = data_to_keep

### Delete specific muscles

In [ ]:
muscles_to_remove = ["VM_LEFT", "VM_RIGHT"]

if False:
    for subject in subject_data.keys():
        for session in subject_data[subject].keys():
            filtered_session_data = subject_data[subject][session]["filtered_session_data"]

            for profile in filtered_session_data["ASSISTED"].keys():
                for emg_df in filtered_session_data["ASSISTED"][profile]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for t_unpowered in filtered_session_data["UNPOWERED"].keys():
                for emg_df in filtered_session_data["UNPOWERED"][t_unpowered]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for side in filtered_session_data["MVIC"].values():
                side.drop(columns=muscles_to_remove, inplace=True, errors="ignore")

### Full data corresponding to 1 iteration per profile

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_37%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

n_rows = len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"])
n_cols = len(filtered_session_data["ASSISTED"].keys())

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"overview_force_muscle_crossed.svg"

target_muscle = "BF"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3, n_rows * 2), sharey='row')
fig.suptitle(f"Subject {subject}, Session {session}\nForce and EMG per profile")

for i, profile in enumerate(filtered_session_data["ASSISTED"].keys()):
    axs[0, i].set_title(profile_names_paper[profile])
    
    for j, motor_df in enumerate(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]):
        emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][j]

        # Plot force values on the primary y-axis
        axs[j, i].plot(motor_df.index, motor_df.F_Y, label=r"$F_Y$", color='orange')
        axs[j, i].tick_params(axis='y', labelcolor='orange')

        # Create a secondary y-axis for EMG data
        ax2 = axs[j, i].twinx()
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle} RIGHT", color='green')
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle} LEFT", color=u'#1f77b4')
        ax2.set_ylim(0, 0.6)  # Set EMG activation scale from 0 to 1

        if j not in data_to_keep.get(profile):
            # Draw a red cross over the specified subplot
            xlim = axs[j, i].get_xlim()
            ylim = axs[j, i].get_ylim()
            axs[j, i].plot(xlim, ylim, color="red", lw=2)  # Diagonal from bottom-left to top-right
            axs[j, i].plot(xlim, ylim[::-1], color="red", lw=2)  # Diagonal from top-left to bottom-right

        if i == len(profiles) - 1:
            ax2.set_ylabel('EMG (% MVIC)', color=u'#1f77b4')
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelcolor=u'#1f77b4')

        handles1, labels1 = axs[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2

        axs[j, 0].set_ylabel(f"Iteration {j+1}\nForce (N)")
        axs[-1, i].set_xlabel("Time (s)")

fig.legend(handles, labels, loc="upper right", ncols=len(handles), bbox_to_anchor=(0.6, 0.93))
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
shutil.copy(filename.with_suffix('.png'), latex_figure_dir / "emg_plots")
plt.show()

In [ ]:
subject_data.pop("XABI", None)

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"] 
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

        data_to_keep = subject_data[subject][session]["data_to_keep"]

        for profile in filtered_session_data["ASSISTED"].keys():
            filtered_session_data["ASSISTED"][profile]["EMG"] = [filtered_session_data["ASSISTED"][profile]["EMG"][i] for i in data_to_keep[profile]]
            filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"] = [filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i] for i in data_to_keep[profile]]
            filtered_session_data["ASSISTED"][profile]["MARKER_DATA"] = [filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] for i in data_to_keep[profile]]

            # Update ranges as well
            relevant_time_ranges[profile]["ranges"] = [relevant_time_ranges[profile]["ranges"][i] for i in data_to_keep[profile]]
            relevant_time_ranges[profile]["hip_vel"] = [relevant_time_ranges[profile]["hip_vel"][i] for i in data_to_keep[profile]]

            assert len(filtered_session_data["ASSISTED"][profile]["EMG"]) == len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]) == len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])

### Delete specific muscles

In [ ]:
muscles_to_remove = ["VM_LEFT", "VM_RIGHT"]

if False:
    for subject in subject_data.keys():
        for session in subject_data[subject].keys():
            filtered_session_data = subject_data[subject][session]["filtered_session_data"]

            for profile in filtered_session_data["ASSISTED"].keys():
                for emg_df in filtered_session_data["ASSISTED"][profile]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for t_unpowered in filtered_session_data["UNPOWERED"].keys():
                for emg_df in filtered_session_data["UNPOWERED"][t_unpowered]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for side in filtered_session_data["MVIC"].values():
                side.drop(columns=muscles_to_remove, inplace=True, errors="ignore")

### Full data corresponding to 1 iteration per profile

In [ ]:
n_prof = len([profile for profile in relevant_time_ranges.keys() if 'unpowered' not in profile.lower()])

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"full_data_overview.svg"
target_muscle = "BF"

fig, axs = plt.subplots(nrows=4, ncols=n_prof, sharex='col', figsize=(20, 5))
fig.suptitle("Overview of subject relevant data\nBeginning and end of iteration highlighted")

for i, (profile, range_list) in enumerate(relevant_time_ranges.items()):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

    if profile.startswith("UNPOWERED"):
        continue

    df_index = 1

    rel_range = range_list["ranges"][df_index] # Get first iteration range

    marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][df_index] 
    motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][df_index]
    emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][df_index]
    
    axs[0, i].set_title(profile_names_paper[profile])
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"], label='Hip Z', color=colors.__next__())
    axs[0, 0].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(rel_range.min, linestyle='--', color="red")
    axs[0, i].axvline(rel_range.max, linestyle='--', color="red")

    axs[1, i].plot(motor_data.index, motor_data["theta_2"], label=r"$\theta_2$", color=colors.__next__())
    axs[1, 0].set_ylabel(r"$\theta_2$ (rad)")
    axs[1, i].axvline(rel_range.min, linestyle='--', color="red")
    axs[1, i].axvline(rel_range.max, linestyle='--', color="red")

    ax2 = axs[1, i].twinx()
    ax2.set_ylim(0, 100)
    ax2.plot(motor_data.index, motor_data.Percentage, label="STS %", color='g', linestyle='--')
    if i == n_prof - 1:
        ax2.set_ylabel('STS %', color='g')
    ax2.tick_params(axis='y', labelcolor='g')

    axs[2, i].plot(motor_data.index, motor_data["F_Y"], label=r"$F_Y$", color=colors.__next__())
    axs[2, 0].set_ylabel(r"$F_Y$ (N)")
    axs[2, i].axvline(rel_range.min, linestyle='--', color="red")
    axs[2, i].axvline(rel_range.max, linestyle='--', color="red")

    axs[3, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle} LEFT", color=colors.__next__())
    axs[3, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle} RIGHT", color=colors.__next__())

    axs[3, i].set_xlabel("Time (s)")
    axs[3, 0].set_ylabel("EMG\n(% of MVIC)")
    axs[3, i].axvline(rel_range.min, linestyle='--', color="red")
    axs[3, i].axvline(rel_range.max, linestyle='--', color="red")

    axs[3, i].axvline(rel_range.min, linestyle='--', color="red")
    axs[3, i].axvline(rel_range.max, linestyle='--', color="red")
    
    # Get the bottom y-limit of the plot for positioning the text
    y_text_position = axs[3, i].get_ylim()[0]
    # A little padding from the bottom edge
    padding = (axs[3, i].get_ylim()[1] - y_text_position) * 0.02

    axs[3, i].text(rel_range.min, y_text_position - padding, r'$t_0$', ha='center', va='top', color="red")
    axs[3, i].text(rel_range.max, y_text_position - padding, r'$t_f$', ha='center', va='top', color="red")
    
    axs[3, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0, symbol=None))


plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.tight_layout()

In [ ]:
profile = "peak_27%_62N"
ranges = relevant_time_ranges[profile]["ranges"]

fig, axs = plt.subplots(3, len(ranges), figsize=(15, 5), sharex='col', sharey='row')
fig.suptitle(f"Profile {profile_names_paper[profile]}")

target_muscle = "BF"

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"discretising_STS_overview_BF.svg"

for i, (emg_df, motor_df, marker_df) in enumerate(zip(*filtered_session_data["ASSISTED"][profile].values())):
    cur_range = ranges[i]

    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

    axs[0, i].set_title(f"Iteration {i+1}")

    axs[0, i].plot(marker_df.index, marker_df["Hip Left Z"], label="Hip Z", color=colors.__next__())
    axs[0, i].axvline(cur_range.min, linestyle='--', color="red")
    axs[0, i].axvline(cur_range.max, linestyle='--', color="red")
    axs[0, 0].set_ylabel(f"MARKER DATA\nHip Z (m)")

    hip_vel = relevant_time_ranges[profile]['hip_vel'][i]
    ax1 = axs[0, i].twinx() 
    ax1_color = colors.__next__()
    ax1.plot(hip_vel.index, hip_vel, label="Hip Vel", color=ax1_color)
    if i == len(ranges) - 1:
        ax1.set_ylabel("Hip Vel (m/s)")
    ax1.tick_params(axis='y', labelcolor=ax1_color)
    ax1.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))

    axs[1, i].plot(motor_df.index, motor_df["F_Y"], label=f"$F_Y$", color=colors.__next__())
    axs[1, 0].set_ylabel(f"MOTOR DATA\n$F_Y (N)$")
    axs[1, i].axvline(cur_range.min, linestyle='--', color="red")
    axs[1, i].axvline(cur_range.max, linestyle='--', color="red")
    
    ax2 = axs[1, i].twinx()
    ax2_color = colors.__next__()
    ax2.plot(motor_df.index, motor_df["Percentage"], label="STS %", color=ax2_color, linestyle='--')
    if i == len(ranges) - 1:
        ax2.set_ylabel('STS %')
    ax2.tick_params(axis='y', labelcolor=ax2_color)
    ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=100))
    ax2.set_ylim(0, 100)

    axs[2, i].plot(emg_df.index, emg_df[f"{target_muscle}_LEFT"], label=f"{target_muscle} LEFT", color=colors.__next__(), linewidth=3)
    axs[2, i].plot(emg_df.index, emg_df[f"{target_muscle}_RIGHT"], label=f"{target_muscle} RIGHT", color=colors.__next__())
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[2, i].axvline(cur_range.min, linestyle='--', color="red")
    axs[2, i].axvline(cur_range.max, linestyle='--', color="red")
    axs[2, 0].set_ylabel("EMG DATA\n(% of MVIC)")
    axs[2, i].set_xlabel("Time (s)")

    handles0, labels0 = axs[0, i].get_legend_handles_labels()
    handles1, labels1 = axs[1, i].get_legend_handles_labels()
    handles2, labels2 = axs[2, i].get_legend_handles_labels()
    handles3, labels3 = ax1.get_legend_handles_labels()
    handles4, labels4 = ax2.get_legend_handles_labels()

    handles = handles0 + handles1 + handles2 + handles3 + handles4
    labels = labels0 + labels1 + labels2 + labels3 + labels4

    y_text_position = axs[2, i].get_ylim()[0]
    # A little padding from the bottom edge
    padding = (axs[2, i].get_ylim()[1] - y_text_position) * 0.02

    axs[2, i].text(cur_range.min, y_text_position - padding, r'$t_0$', ha='center', va='top', color="red")
    axs[2, i].text(cur_range.max, y_text_position - padding, r'$t_f$', ha='center', va='top', color="red")
    
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None, decimals=0))
    axs[2, i].set_ylim(0, 0.6)

fig.legend(handles, labels, loc="upper center", ncol=len(labels), bbox_to_anchor=(0.5, 0.95))
plt.tight_layout(rect=(0, 0, 1, 0.95))
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Trim all dataframes based on relevant TIME window

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]

        trimmed_time_dfs = {"ASSISTED": dict(), "UNPOWERED": dict()}

        for profile in filtered_session_data["ASSISTED"].keys():
            rel_ranges = relevant_time_ranges[profile]["ranges"] # Get first iteration range
            trimmed_time_dfs["ASSISTED"][profile] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["ASSISTED"][profile].values()


            trimmed_time_dfs["ASSISTED"][profile]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
            trimmed_time_dfs["ASSISTED"][profile]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]

            for motor_df, rel_range in zip(motor_dfs, rel_ranges):
                if rel_range.min > motor_df.index[-1]:
                    print("Warning: cut-off range starts after motor data. Keeping whole range")
                    rel_range = rel_range._replace(min=motor_df.index[0])
                # trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"].append(motor_df.loc[rel_range.min:rel_range.max])

        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            rel_ranges = relevant_time_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"] # Get first iteration range

            trimmed_time_dfs["UNPOWERED"][t_unpowered] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["UNPOWERED"][t_unpowered].values()

            trimmed_time_dfs["UNPOWERED"][t_unpowered]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            trimmed_time_dfs["UNPOWERED"][t_unpowered]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]
            trimmed_time_dfs["UNPOWERED"][t_unpowered]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
        
        subject_data[subject][session]["trimmed_time_dfs"] = trimmed_time_dfs

## Discretize STS from time to phase percentage

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]

        for profile in trimmed_time_dfs["ASSISTED"].keys():
            emg, motor, marker = trimmed_time_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)

                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100

                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

        for unpowered_moment in trimmed_time_dfs['UNPOWERED'].keys():
            emg, motor, marker = trimmed_time_dfs['UNPOWERED'][unpowered_moment].values()
            
            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)
                
                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100


                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

### Align force peak with peak timing percentage

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]
                
        shift_motor_data = deepcopy(relevant_time_ranges)
        profile_peaks = {profile: int(profile.split("_")[1][:-1]) for profile in trimmed_time_dfs["ASSISTED"].keys()}

        for profile in trimmed_time_dfs["ASSISTED"].keys():
            for i, motor_df in enumerate(trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"]):
                old_range = relevant_time_ranges[profile]["ranges"][i]

                max_ind = motor_df.F_Y.argmax()
                old_peak_time = motor_df.iloc[max_ind].time

                closest_index_value = abs(motor_df.index - profile_infos[profile]["peak_time"]).argmin()
                closest_t_ind = motor_df.index[closest_index_value]
                new_peak_time = motor_df.loc[closest_t_ind].time

                shift_by = new_peak_time - old_peak_time

                new_range = time_range(old_range.min - shift_by, old_range.max - shift_by)
                shift_motor_data[profile]["ranges"][i] = new_range

        subject_data[subject][session]["time_shifted_motor_ranges"] = shift_motor_data

### Reset index to account for shifting

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        relevant_time_ranges = subject_data[subject][session]["relevant_time_ranges"]
        time_shifted_motor_ranges = subject_data[subject][session]["time_shifted_motor_ranges"]

        trimmed_time_dfs = {"ASSISTED": dict(), "UNPOWERED": dict()}

        for profile in filtered_session_data["ASSISTED"].keys():
            rel_ranges = relevant_time_ranges[profile]["ranges"] # Get first iteration range
            motor_ranges = time_shifted_motor_ranges[profile]["ranges"]
            trimmed_time_dfs["ASSISTED"][profile] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["ASSISTED"][profile].values()

            trimmed_time_dfs["ASSISTED"][profile]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, motor_ranges)]
            trimmed_time_dfs["ASSISTED"][profile]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]

            for motor_df, rel_range in zip(motor_dfs, rel_ranges):
                if rel_range.min > motor_df.index[-1]:
                    print("Warning: cut-off range starts after motor data. Keeping whole range")
                    rel_range = rel_range._replace(min=motor_df.index[0])
                # trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"].append(motor_df.loc[rel_range.min:rel_range.max])

        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            rel_ranges = relevant_time_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"] # Get first iteration range

            trimmed_time_dfs["UNPOWERED"][t_unpowered] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["UNPOWERED"][t_unpowered].values()

            trimmed_time_dfs["UNPOWERED"][t_unpowered]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            trimmed_time_dfs["UNPOWERED"][t_unpowered]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]
            trimmed_time_dfs["UNPOWERED"][t_unpowered]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
        
        subject_data[subject][session]["trimmed_time_dfs"] = trimmed_time_dfs

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]

        for profile in trimmed_time_dfs["ASSISTED"].keys():
            emg, motor, marker = trimmed_time_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)

                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100

                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

        for unpowered_moment in trimmed_time_dfs['UNPOWERED'].keys():
            emg, motor, marker = trimmed_time_dfs['UNPOWERED'][unpowered_moment].values()
            
            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)
                
                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100


                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

### Plotting the time-shifted data

In [ ]:
subject = "MIH02"
session = "December_20"

trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

profiles = trimmed_time_dfs["ASSISTED"].keys()

n_rows = len(trimmed_time_dfs["ASSISTED"]["peak_77%_62N"]["MOTOR_DATA"])
n_cols = len(profiles)

target_muscle = "BF"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 7), sharex=True, sharey='row')  # Adjust as needed
fig.suptitle(f"{subject}\nAssisted activations for {target_muscle}\n")

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"motor_emg_{target_muscle}_assisted.svg"

for col, profile in enumerate(profiles):
    for row in range(len(trimmed_time_dfs["ASSISTED"][profile]["MARKER_DATA"])):
        motor_df = trimmed_time_dfs["ASSISTED"][profile]["MOTOR_DATA"][row]  # Adjusted for demonstration
        emg_df = trimmed_time_dfs["ASSISTED"][profile]["EMG"][row]  # Adjusted for demonstration
        cur_range = relevant_time_ranges[profile]["ranges"][row]  # Assuming relevant_time_ranges is structured similarly
        hip_vel = relevant_time_ranges[profile]["hip_vel"][row]  # Assuming relevant_time_ranges is structured similarly

        axs[row, col].plot(motor_df.index, motor_df.F_Y, label=r"$F_Y$", color="g")
        axs[row, col].set_ylim(0, 70)
        axs[row, col].tick_params(axis='y', labelcolor='g')

        ax2 = axs[row, col].twinx()
        ax2.plot(emg_df.index, emg_df[f"{target_muscle}_LEFT"], label="LEFT")
        ax2.plot(emg_df.index, emg_df[f"{target_muscle}_RIGHT"], label="RIGHT")
        ax2.set_yticklabels('')
        if col == n_cols - 1:
            # Add N to the right y-axis
            ax2.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x * 100:.0f}'))

        if col == len(profiles) - 1:
            ax2.set_ylabel("EMG (% MVIC)")
        ax2.set_ylim(0, 1)  # Set EMG activation scale from 0 to 1

        handles1, labels1 = axs[row, col].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2

        axs[row, 0].set_ylabel(f"Iteration {row+1}\nForce (N)")
        axs[-1, col].set_xlabel("STS (%")

        axs[0, col].set_title(profile_names_paper[profile], fontsize=16)

fig.legend(handles, labels, loc="upper center", ncol=len(handles), bbox_to_anchor=(0.5, 0.9))

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

## Extract metrics from EMG data

## Extract metrics from EMG data

### Average activation signals across trials

#### Equalize length of MOTOR and EMG data across iterations

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]        

        interpolated_dfs = deepcopy(trimmed_time_dfs)

        for profile in trimmed_time_dfs["ASSISTED"].keys():
            emg, motor, marker = trimmed_time_dfs["ASSISTED"][profile].values()

            emg_len = min([len(emg_df) for emg_df in emg])
            motor_len = min([len(motor_df) for motor_df in motor])
            marker_len = min([len(marker_df) for marker_df in marker])

            for i in range(len(emg)):
                interpolated_dfs["ASSISTED"][profile]["EMG"][i] = interpolate_dataframe_to_length(emg[i], emg_len)
                interpolated_dfs["ASSISTED"][profile]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], motor_len)
                interpolated_dfs["ASSISTED"][profile]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], marker_len)
                
        for unpowered_time in trimmed_time_dfs["UNPOWERED"].keys():
            emg, motor, marker = trimmed_time_dfs["UNPOWERED"][unpowered_time].values()

            emg_len = min([len(emg_df) for emg_df in emg])
            motor_len = min([len(motor_df) for motor_df in motor])
            marker_len = min([len(marker_df) for marker_df in marker])

            for i in range(len(emg)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["EMG"][i] = interpolate_dataframe_to_length(emg[i], emg_len)
                interpolated_dfs["UNPOWERED"][unpowered_time]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], motor_len)
                interpolated_dfs["UNPOWERED"][unpowered_time]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], marker_len)
        
        subject_data[subject][session]["interpolated_dfs"] = interpolated_dfs

### Average activations and motor data

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]

        interpolated_dfs = deepcopy(trimmed_time_dfs)

        # Initialize variables to store the minimum lengths for each type of data
        min_emg_len = float('inf')
        min_motor_len = float('inf')
        min_marker_len = float('inf')

        # Step 1: Identify the Minimum Lengths across all "ASSISTED" profiles
        for profile in trimmed_time_dfs["ASSISTED"].keys():
            emg, motor, marker = trimmed_time_dfs["ASSISTED"][profile].values()

            min_emg_len = min(min_emg_len, min([len(emg_df) for emg_df in emg]))
            min_motor_len = min(min_motor_len, min([len(motor_df) for motor_df in motor]))
            min_marker_len = min(min_marker_len, min([len(marker_df) for marker_df in marker]))

        # Step 2: Apply Interpolation to "ASSISTED"
        for profile in trimmed_time_dfs["ASSISTED"].keys():
            emg, motor, marker = trimmed_time_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                interpolated_dfs["ASSISTED"][profile]["EMG"][i] = interpolate_dataframe_to_length(emg[i], min_emg_len)
            for i in range(len(motor)):
                interpolated_dfs["ASSISTED"][profile]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], min_motor_len)
            for i in range(len(marker)):
                interpolated_dfs["ASSISTED"][profile]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], min_marker_len)

        # Step 3: Apply the Same Interpolation to "UNPOWERED"
        for unpowered_time in trimmed_time_dfs["UNPOWERED"].keys():
            emg, motor, marker = trimmed_time_dfs["UNPOWERED"][unpowered_time].values()

            for i in range(len(emg)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["EMG"][i] = interpolate_dataframe_to_length(emg[i], min_emg_len)
            for i in range(len(motor)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], min_motor_len)
            for i in range(len(marker)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], min_marker_len)
        
        subject_data[subject][session]["interpolated_dfs"] = interpolated_dfs

In [ ]:
# Drop time column
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        interpolated_dfs = subject_data[subject][session]["interpolated_dfs"]
        for profile in interpolated_dfs["ASSISTED"].keys():
            for emg_df, motor_df, marker_df in zip(*interpolated_dfs["ASSISTED"][profile].values()):
                emg_df.drop(columns="TIME", inplace=True, errors='ignore')
                marker_df.drop(columns="TIME", inplace=True, errors='ignore')

        for unpowered_time in interpolated_dfs["UNPOWERED"].keys():
            for emg_df, motor_df, marker_df in zip(*interpolated_dfs["UNPOWERED"][unpowered_time].values()):
                emg_df.drop(columns="TIME", inplace=True, errors='ignore')
                marker_df.drop(columns="TIME", inplace=True, errors='ignore')

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        interpolated_dfs = subject_data[subject][session]["interpolated_dfs"]
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
                
        average_profile_dfs = {"ASSISTED": {}, "UNPOWERED": {}}
        muscles = interpolated_dfs["UNPOWERED"]["BEFORE"]["EMG"][0].columns

        motor_assisted_columns = interpolated_dfs["ASSISTED"][list(interpolated_dfs["ASSISTED"].keys())[0]]["MOTOR_DATA"][0].columns
        motor_unpowered_columns = interpolated_dfs["UNPOWERED"]["BEFORE"]["MOTOR_DATA"][0].columns

        for mode, columns in zip(["ASSISTED", "UNPOWERED"], [motor_assisted_columns, motor_unpowered_columns]):
            for profile in interpolated_dfs[mode].keys():
                average_profile_dfs[mode][profile] = {"EMG": {"mean": None, "std": None}, "MOTOR_DATA": {"mean": None, "std": None}}

                # EMG Data
                muscles_means = {}
                muscles_std = {}

                for emg in interpolated_dfs[mode][profile]["EMG"]:
                    for muscle in emg.columns:
                        if muscle not in muscles_means:
                            muscles_means[muscle] = []
                            muscles_std[muscle] = []
                        muscles_means[muscle].append(emg[muscle])
                        muscles_std[muscle].append(emg[muscle])

                for muscle, dfs in muscles_means.items():
                    merged = pd.concat(dfs, axis=1)
                    muscles_means[muscle] = merged.mean(axis=1)
                    muscles_std[muscle] = merged.std(axis=1)

                average_profile_dfs[mode][profile]["EMG"]["mean"] = smooth_dataframe(pd.DataFrame.from_dict(muscles_means), window_size=1)
                average_profile_dfs[mode][profile]["EMG"]["std"] = smooth_dataframe(pd.DataFrame.from_dict(muscles_std), window_size=1)

                # MOTOR Data
                motor_means = {}
                motor_std = {}

                for motor_data in interpolated_dfs[mode][profile]["MOTOR_DATA"]:
                    for column in columns:
                        if column not in motor_means:
                            motor_means[column] = []
                            motor_std[column] = []
                        motor_means[column].append(motor_data[column])
                        motor_std[column].append(motor_data[column])

                for column, dfs in motor_means.items():
                    merged = pd.concat(dfs, axis=1)
                    motor_means[column] = merged.mean(axis=1)
                    motor_std[column] = merged.std(axis=1)

                average_profile_dfs[mode][profile]["MOTOR_DATA"]["mean"] = smooth_dataframe(pd.DataFrame.from_dict(motor_means))
                average_profile_dfs[mode][profile]["MOTOR_DATA"]["std"] = smooth_dataframe(pd.DataFrame.from_dict(motor_std))

        average_profile_dfs["UNPOWERED"]["BASELINE"] = {"EMG": {"mean": None, "std": None}, "MOTOR_DATA": {"mean": None, "std": None}}

        if "AFTER" in filtered_session_data["UNPOWERED"].keys():
            average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = (average_profile_dfs["UNPOWERED"]["BEFORE"]["EMG"]["mean"] + average_profile_dfs["UNPOWERED"]["AFTER"]["EMG"]["mean"]) / 2
            average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"] = (average_profile_dfs["UNPOWERED"]["BEFORE"]["EMG"]["std"] + average_profile_dfs["UNPOWERED"]["AFTER"]["EMG"]["std"]) / 2
        else:
            average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = average_profile_dfs["UNPOWERED"]["BEFORE"]["EMG"]["mean"]
            average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"] = average_profile_dfs["UNPOWERED"]["BEFORE"]["EMG"]["std"]
        
        subject_data[subject][session]["average_profile_dfs"] = average_profile_dfs

In [ ]:
subject = "MIH02"
session = "December_20"

average_profile_dfs = subject_data[subject][session]["average_profile_dfs"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["session_data"])

# Since first column is TIME we ignore it
n_muscles = len(average_profile_dfs["ASSISTED"]["peak_27%_62N"]["EMG"]["mean"].columns)

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"full_EMG_activations_and_motor.svg"

# We add 1 because we want to include the motor data in the first row
fig, ax = plt.subplots(nrows=n_muscles + 1, ncols=n_prof, figsize=(30, 20))

if enable_paper_scaling:
    fig.suptitle(f"{subject}\nEMG activations (paper match scaling)", fontsize=30)
else:
    fig.suptitle(f"{subject}\nEMG activations (MVIC scaled)", fontsize=30)

for i, profile in enumerate(average_profile_dfs["ASSISTED"].keys()):
    force_data = average_profile_dfs["ASSISTED"][profile]["MOTOR_DATA"]

    
    closest_index_value = abs(force_data["mean"].index - profile_infos[profile]["peak_time"]).argmin()
    closest_t_ind = force_data["mean"].index[closest_index_value]

    ax[0, i].set_title(profile_names_paper[profile], fontsize=20)
    ax[0, i].plot(force_data["mean"].index, force_data["mean"].F_Y, label=r"$F_Y$", color='green')
    ax[0, i].axvline(closest_t_ind, color="red")
    ax[0, i].fill_between(force_data["mean"].index, force_data["mean"].F_Y - force_data["std"].F_Y, force_data["mean"].F_Y + force_data["std"].F_Y, alpha=0.2, color='green')
    ax[0, 0].set_ylabel('Force (N)', color='green', fontsize=16)
    ax[0, i].tick_params(axis='y', labelcolor='green', labelsize=16)
    ax[0, i].yaxis.set_major_formatter("{x:.0f}N")

    ax2 = ax[0, i].twinx()
    ax2.plot(force_data["mean"].index, force_data["mean"].Percentage/100, label="STS %", linestyle="--", color="purple")
    ax2.fill_between(force_data["mean"].index, force_data["mean"].Percentage/100 - force_data["std"].Percentage/100, force_data["mean"].Percentage/100 + force_data["std"].Percentage/100, alpha=0.2, color='purple')
    if i ==len(average_profile_dfs["ASSISTED"].keys())-1:
        ax2.set_ylabel('Measured STS %', color='purple', fontsize=16)
    ax2.tick_params(axis='y', labelcolor='purple', labelsize=16)
    ax2.xaxis.set_major_formatter(mticker.PercentFormatter())
    ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    ax2.set_ylim(0, 1)

    for j in range(1, n_muscles + 1):
        muscle = average_profile_dfs["ASSISTED"][profile]["EMG"]["mean"].columns[j-1]
        peak_time = profile_infos[profile]["peak_time"]
        
        if i == 0:
            ax[j, i].set_ylabel(f"{muscle}\nEMG (% of MVIC)", fontsize=16)

        ax[-1, i].set_xlabel("Percentage of STS", fontsize=16)

        emg_mean = average_profile_dfs["ASSISTED"][profile]["EMG"]["mean"][muscle]
        emg_std = average_profile_dfs["ASSISTED"][profile]["EMG"]["std"][muscle]

        closest_peak_index = np.abs(emg_mean.index.astype(float) - peak_time).argmin()

        emg_baseline_mean = average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
        emg_baseline_std = average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]
           

        ax[j, i].plot(emg_mean.index, emg_mean, label=f"Trials")
        ax[j, i].fill_between(emg_mean.index, emg_mean - emg_std, emg_mean + emg_std, alpha=0.2)
        ax[j, i].plot(emg_baseline_mean.index, emg_baseline_mean, label=f"Baseline")
        ax[j, i].fill_between(emg_baseline_mean.index, emg_baseline_mean - emg_baseline_std, emg_baseline_mean + emg_baseline_std, alpha=0.2)
        ax[j, i].tick_params(axis='x', labelsize=16)
        ax[j, i].tick_params(axis='y', labelsize=16)
        ax[j, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
        ax[j, i].set_ylim(0, 1)
        
        force_handles, force_labels = ax[0, i].get_legend_handles_labels()

        handles1, labels1 = ax[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2 + force_handles
        
        labels = labels1 + labels2 + force_labels
    
legend = fig.legend(handles, labels, loc="upper center", ncols=len(labels), fontsize=20, bbox_to_anchor=(0.5, 0.93))

for line in legend.get_lines():
    line.set_linewidth(4)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, dpi=500, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
shutil.copy(filename, latex_figure_dir / "emg_plots")
plt.show()

### Calculate mean activation delta

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        average_profile_dfs = subject_data[subject][session]["average_profile_dfs"]      

        averaged_activations = deepcopy(average_profile_dfs)

        for profile in average_profile_dfs["ASSISTED"].keys():
            averaged_activations["ASSISTED"][profile]["EMG"]["mean"] = average_profile_dfs["ASSISTED"][profile]["EMG"]["mean"].mean()
            averaged_activations["ASSISTED"][profile]["EMG"]["std"] = average_profile_dfs["ASSISTED"][profile]["EMG"]["std"].mean()


        averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"].mean()
        averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"] = average_profile_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"].mean()

        averaged_activations['UNPOWERED'].pop("BEFORE", None)
        averaged_activations['UNPOWERED'].pop("AFTER", None)

        subject_data[subject][session]["averaged_activations"] = averaged_activations

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        averaged_activations = subject_data[subject][session]["averaged_activations"]

        n_profiles = len(averaged_activations["ASSISTED"].keys())
        muscles = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"].index
        
        muscle_deltas = []  # For storing deltas
        muscle_activations = []  # For storing activations

        for muscle in muscles:
            mean_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
            std_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]
            
            muscle_activations.append({"Profile": "BASELINE", "Muscle": muscle, "Mean Activation": mean_baseline, "STD Activation": std_baseline})


        for profile in profiles:
            muscle_means_delta = []  # To calculate deltas
            muscle_stds_delta = []

            muscle_means_activation = []  # To store activations
            muscle_stds_activation = []

            for muscle in muscles:
                mean_assisted = averaged_activations["ASSISTED"][profile]["EMG"]["mean"][muscle]
                std_assisted = averaged_activations["ASSISTED"][profile]["EMG"]["std"][muscle]
                
                mean_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
                std_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]

                # Calculate deltas
                mean_delta = (mean_assisted - mean_baseline) / mean_baseline
                std_delta = (std_assisted + std_baseline)
                
                # Append delta calculations
                muscle_means_delta.append(mean_delta)
                muscle_stds_delta.append(std_delta)
                muscle_deltas.append({"Profile": profile, "Muscle": muscle, "Mean Delta": mean_delta, "STD Delta": std_delta})
                
                # Append activations
                muscle_means_activation.append(mean_assisted)
                muscle_stds_activation.append(std_assisted)
                muscle_activations.append({"Profile": profile, "Muscle": muscle, "Mean Activation": mean_assisted, "STD Activation": std_assisted})
                
            # Append average of all muscles to each profile for deltas
            muscles_delta_df = pd.DataFrame({"means": muscle_means_delta, "stds": muscle_stds_delta}, index=muscles).mean()
            muscle_deltas.append({"Profile": profile, "Muscle": "AVERAGE", "Mean Delta": muscles_delta_df.means, "STD Delta": muscles_delta_df.stds})
            
            # Append average of all muscles to each profile for activations
            muscles_activation_df = pd.DataFrame({"means": muscle_means_activation, "stds": muscle_stds_activation}, index=muscles).mean()
            muscle_activations.append({"Profile": profile, "Muscle": "AVERAGE", "Mean Activation": muscles_activation_df.means, "STD Activation": muscles_activation_df.stds})
        
            
        # Convert the collected data into DataFrames and store them under their respective keys
        df_deltas = pd.DataFrame(muscle_deltas)
        df_activations = pd.DataFrame(muscle_activations)
        subject_data[subject][session]["mean_delta_activations"] = df_deltas
        subject_data[subject][session]["absolute_mean_activations"] = df_activations

In [ ]:
fig, axs = plt.subplots(figsize=(10, 5))
fig.suptitle(f"Mean muscle activation $\Delta$\nper subject and profile", fontsize=20)

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"mean_delta_activations.svg"

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"mean_delta_activations.svg"

profile_deltas = {}

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_delta_activations = subject_data[subject][session]["mean_delta_activations"]
        muscle_averages = mean_delta_activations[mean_delta_activations["Muscle"] == "AVERAGE"]
        
        muscle_averages.loc[:, "Profile"] = list(profile_peaks.values())

        axs.plot(muscle_averages["Profile"], muscle_averages["Mean Delta"], label=subject, marker='*', linewidth=3, markersize=10)

        for index, row in muscle_averages.iterrows():
            if row["Profile"] not in profile_deltas:
                profile_deltas[row["Profile"]] = []
            profile_deltas[row["Profile"]].append(row["Mean Delta"])

average_deltas = {profile: np.mean(deltas) for profile, deltas in profile_deltas.items()}
average_values = list(average_deltas.values())

axs.grid(False)
axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
axs.hlines(0, profile_peaks['peak_27%_62N'], profile_peaks['peak_77%_62N'], color='black', linestyle='--')
axs.set_xlabel("Peak Timing of Assistance (%)")
axs.set_xticks(list(profile_peaks.values()))
axs.set_ylabel("Muscle Activity Reduction (%)")

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)
# axs.spines['bottom'].set_visible(False)  # Adjust thickness of the arrow

# Make x-axis and y-axis arrows
axs.spines['left'].set_linewidth(2)
axs.spines['bottom'].set_linewidth(2)  # Adjust thickness of the arrow
axs.spines['bottom'].set_capstyle('projecting')  # Arrow style
offset = 0.03


axs.set_ylim(-0.4, 0.2)
axs.legend(fontsize=16, title="Subjects", title_fontsize='16', loc='upper right')

# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(filename, format='svg')
plt.savefig("../paper_figures/mean_delta_activations.svg", format='svg')
plt.savefig("../paper_figures/mean_delta_activations.png", dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / "mean_delta_activations.png", dpi=500, format="png")
plt.show()



In [ ]:
# Plot average muscle activations per profile as a line plot
n_subjects = len(subject_data.keys())


fig, axs = plt.subplots(1, n_subjects, figsize=(20, 5), sharey=True)

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        mean_activations = subject_data[subject][session]["absolute_mean_activations"]
        mean_activations = mean_activations[mean_activations["Profile"] != "BASELINE"]
        for muscle in mean_activations.Muscle.unique():
            muscle_averages = mean_activations[mean_activations["Muscle"] == muscle]

            muscle_averages.loc[:, "Profile"] = list(profile_peaks.values())

            axs[i].plot(muscle_averages["Profile"], muscle_averages["Mean Activation"], label=muscle, marker='*', linewidth=3, markersize=10)

        axs[i].set_xticks(list(profile_peaks.values()))
        axs[i].set_title(f"Subject {subject}", fontsize=20)
        axs[i].legend(loc="upper right", fontsize=10)
        axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
        axs[i].set_xlabel("Peak Timing of Assistance (%)")
        axs[0].set_ylabel("Mean Activation (% of MVIC)")

plt.savefig("../paper_figures/mean_absolute_activations_per_subject.svg", format='svg')
plt.savefig("../paper_figures/mean_absolute_activations_per_subject.png", dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / 'mean_absolute_activations_per_subject.png', dpi=500, format="png")
plt.show()

In [ ]:
n_subjects = len(subject_data.keys())

fig, axs = plt.subplots(1, n_subjects, figsize=(20, 5), sharey=True)  # Ensure axs is 2D
# Predefined colors for specific muscles
colors = {'BASELINE': '#1f77b4', 'AVERAGE': 'orange'}

for i, subject in enumerate(subject_data.keys()):
    muscle_colors = {}  # Dictionary to dynamically assign colors to each muscle
    for session in subject_data[subject].keys():
        mean_activations = subject_data[subject][session]['absolute_mean_activations']
        mean_activations = mean_activations[mean_activations["Profile"] != "BASELINE"]
        muscles = sorted(mean_activations.Muscle.unique())
        
        # Dynamically generate grey shades based on the number of muscles
        grey_count = len([m for m in muscles if m not in colors])
        grey_values = np.linspace(0.8, 0.2, grey_count)  # Light (0.8) to dark (0.2)
        grey_counter = 0
        # Assign grey shades to muscles not in the predefined colors
        for index, muscle in enumerate(muscles):
            if muscle not in colors:
                # Use the grey_counter to access grey_values
                grey = grey_values[grey_counter]
                # Convert grey value to RGB format
                rgb_grey = (grey, grey, grey)
                muscle_colors[muscle] = rgb_grey
                # Increment the grey_counter
                grey_counter += 1
        
        # Update the colors dictionary with dynamically generated grey shades
        colors.update(muscle_colors)
        
        for muscle in muscles:
            muscle_averages = mean_activations[mean_activations["Muscle"] == muscle]
            muscle_averages.loc[:, "Profile"] = list(profile_peaks.values())
            color = colors.get(muscle)
            
            # Plot line with dynamically assigned color
            axs[i].plot(muscle_averages["Profile"], muscle_averages["Mean Activation"], 
                           label=muscle, marker='*', linewidth=3, markersize=10, color=color)
            
            # Place muscle name next to the line on the right
            if not muscle_averages.empty:
                axs[i].text(muscle_averages["Profile"].iloc[-1], muscle_averages["Mean Activation"].iloc[-1], 
                               f"  {muscle}", verticalalignment='center', color=color)

        axs[i].set_xticks(list(profile_peaks.values()))
        axs[i].set_title(f"Subject {subject}", fontsize=20)
        axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
        axs[i].set_xlabel("Peak Timing of Assistance (%)")
        axs[0].set_ylabel("Mean Activation\n(% of MVIC)")

# Remove the legend since muscle names are annotated on the plot
for ax in axs.flat:
    ax.legend().set_visible(False)

plt.tight_layout()
plt.savefig("../paper_figures/mean_absolute_activations_per_subject_bw.svg", format='svg')
plt.savefig("../paper_figures/mean_absolute_activations_per_subject_bw.png", dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / 'mean_absolute_activations_per_subject_bw.png', dpi=500, format="png")
plt.show()

In [ ]:
all_deltas = []
all_means = []


for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        df_delta = subject_data[subject][session]["mean_delta_activations"]
        df_means = subject_data[subject][session]["absolute_mean_activations"]
        df_baseline = subject_data[subject][session]["average_profile_dfs"]["UNPOWERED"]["BASELINE"]["EMG"]["mean"]
        
        # Ensuring each DataFrame has 'Subject' and 'Session'
        df_delta['Subject'] = subject
        df_delta['Session'] = session
        df_means['Subject'] = subject
        df_means['Session'] = session
        
        all_deltas.append(df_delta)
        all_means.append(df_means)

df_all_deltas = pd.concat(all_deltas)
df_all_means = pd.concat(all_means)

df_all_deltas.sort_values(by="Muscle", ascending=True, inplace=True)
df_all_means.sort_values(by="Muscle", ascending=True, inplace=True)

In [ ]:
# Create a figure and axis for the plot
plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = Path("../paper_figures") / f"mean_delta_activation_muscle_profile_subject.svg"

fig, ax = plt.subplots(1, n_profiles, figsize=(35, 10), sharey=True)
fig.suptitle("Average Activation $\Delta$\nacross Subjects and Muscles", fontsize=24, y=1.1)


for i, profile in enumerate(sorted(profile_infos.keys())):
    df_deltas_profile = df_all_deltas[df_all_deltas['Profile'] == profile].sort_values(by="Subject")
    df_means_profile = df_all_means[df_all_means['Profile'] == profile].sort_values(by="Subject")

    df_deltas_profile[df_deltas_profile['Muscle'] == "AVERAGE"]

    # Plotting delta activations with consideration for missing data
    sns.barplot(x="Muscle", y="Mean Delta", hue="Subject", data=df_deltas_profile, ax=ax[i], dodge=True)

    # # Adding error bars for delta activations
    ax[i].set_xlabel("Muscles", fontsize=16)
    ax[i].tick_params(axis='x', labelrotation=45, labelsize=16)
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].set_ylabel("Mean Activation $\Delta$ (%)", fontsize=20)
    ax[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    ax[i].set_title(profile_names_paper[profile], fontsize=24)
    ax[i].tick_params(labelleft=True)

    handles, labels = ax[i].get_legend_handles_labels()

    ax[i].get_legend().remove()

    
fig.legend(handles, labels, fontsize=20, ncols=len(handles), bbox_to_anchor=(0.56, 1.02))  
plt.tight_layout()
plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

In [ ]:
subject = "MIH02"
session = "December_20"

df_delta = subject_data[subject][session]["mean_delta_activations"]
df_means = subject_data[subject][session]["absolute_mean_activations"]

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"mean_delta_activation_muscle_profile_{subject}.svg"

fig, axs = plt.subplots(1, n_profiles, figsize=(30, 10))
fig.suptitle(f"Subject {subject}\nMean muscle activation $\Delta$", fontsize=20, y=1.02)

for i, profile in enumerate(profiles):
    profile_df = df_delta[df_delta["Profile"] == profile]
    sns.barplot(x="Muscle", y="Mean Delta", data=profile_df, ax=axs[i], dodge=False)
    
    # Adding error bars
    for j, muscle in enumerate(profile_df["Muscle"].unique()):
        axs[i].errorbar(j, profile_df[profile_df["Muscle"] == muscle]["Mean Delta"].values[0],
                        yerr=profile_df[profile_df["Muscle"] == muscle]["STD Delta"].values[0], 
                        fmt='none', color='gray', capsize=5, elinewidth=2)
    axs[i].set_title(profile, fontsize=14)
    axs[i].set_xlabel("Muscle", fontsize=16)
    axs[i].set_ylabel("Mean activation delta (%)", fontsize=16)
    axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))

    plt.setp(axs[i].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout(rect=[0, 0, 1, 1])  # Adjust the layout to make room for the suptitle
plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.show()

### Calculate peak activation delta

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        n_profiles = len(averaged_activations["ASSISTED"].keys())
        muscles = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"].index
        data = []

        for profile in profiles:
            muscle_means = []
            muscle_stds = []

            for muscle in muscles:
                mean_assisted = averaged_activations["ASSISTED"][profile]["EMG"]["mean"][muscle]
                std_assisted = averaged_activations["ASSISTED"][profile]["EMG"]["std"][muscle]
                
                mean_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
                std_baseline = averaged_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]

                mean_delta = (mean_assisted - mean_baseline) / mean_baseline
                std_delta = (std_assisted + std_baseline) #/ std_baseline

                muscle_means.append(mean_delta)
                muscle_stds.append(std_delta)
                
                data.append({"Profile": profile, "Muscle": muscle, "Mean Delta": mean_delta, "STD Delta": std_delta})
                
            muscles_delta_df = pd.DataFrame({"means": muscle_means, "stds": muscle_stds}, index=muscles).mean()
            data.append({"Profile": profile, "Muscle": "Average", "Mean Delta": muscles_delta_df.means, "STD Delta": muscles_delta_df.stds})
            # Append average of all muscles to each profile
        df = pd.DataFrame(data)

        subject_data[subject][session]["mean_delta_activations"] = df

In [ ]:
# Calculate peak EMG values for each muscle (before and after trials)
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        trimmed_time_dfs = subject_data[subject][session]["trimmed_time_dfs"]
        muscles = subject_data[subject][session]["averaged_activations"]["ASSISTED"]["peak_77%_62N"]["EMG"]["mean"].index

        peak_emg_means = []
        
        for t_unpowered in trimmed_time_dfs["UNPOWERED"].keys():
            peak_means = [df.max() for df in trimmed_time_dfs["UNPOWERED"][t_unpowered]["EMG"]]
            peak_emg_means.append(sum(peak_means) / len(peak_means))

        peak_emg_means = sum(peak_emg_means) / len(peak_emg_means)

        # Calculate average peak EMG values for each muscle (assisted trials)
        peak_values_assisted = defaultdict(dict)
        for profile in trimmed_time_dfs["ASSISTED"].keys():
            max_values_assisted = pd.DataFrame([df.max() for df in trimmed_time_dfs["ASSISTED"][profile]["EMG"]])
            peak_values_assisted[profile]["mean"] = max_values_assisted.mean()
            peak_values_assisted[profile]["std"] = max_values_assisted.std()

        peak_activation_deltas = []

        for profile in profiles:
            peak_means = []
            peak_stds = []

            for muscle in muscles:
                peak_mean = peak_values_assisted[profile]["mean"][muscle]
                peak_std = peak_values_assisted[profile]["std"][muscle]

                peak_mean_baseline = peak_emg_means[muscle]
                peak_std_baseline = peak_emg_means[muscle]

                peak_delta = (peak_mean - peak_mean_baseline) / peak_mean_baseline
                peak_std_delta = (peak_std + peak_std_baseline)


                peak_means.append(peak_delta)
                peak_stds.append(peak_std_delta)

                peak_activation_deltas.append({"Profile": profile, "Muscle": muscle, "Mean Delta": peak_delta, "STD Delta": peak_std_delta})

            peaks_df = pd.DataFrame({"means": peak_means, "stds": peak_stds}, index=muscles).mean()
            peak_activation_deltas.append({"Profile": profile, "Muscle": "Average", "Mean Delta": peaks_df.means, "STD Delta": peaks_df.stds})

        peak_activation_deltas_df = pd.DataFrame(peak_activation_deltas)

        subject_data[subject][session]["peak_activation_deltas"] = peak_activation_deltas_df

In [ ]:
all_deltas = []
all_means = []

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        df_delta = subject_data[subject][session]["peak_activation_deltas"]
        
        # Ensuring each DataFrame has 'Subject' and 'Session'
        df_delta['Subject'] = subject
        df_delta['Session'] = session
        
        all_deltas.append(df_delta)

df_all_deltas = pd.concat(all_deltas)

df_all_deltas.sort_values(by="Muscle", ascending=True, inplace=True)

In [ ]:
# Create a figure and axis for the plot
plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = Path("../paper_figures") / f"average_peak_activation_delta_profile_subject.svg"

fig, ax = plt.subplots(1, n_profiles, figsize=(35, 10), sharey=True)
fig.suptitle("Average Peak Activation $\Delta$\nacross Subjects and Muscles", fontsize=24, y=1.1)


for i, profile in enumerate(sorted(profile_infos.keys())):
    df_peak_deltas = df_all_deltas[df_all_deltas["Profile"] == profile].sort_values(by="Subject")

    # Plotting delta activations with consideration for missing data
    sns.barplot(x="Muscle", y="Mean Delta", hue="Subject", data=df_peak_deltas, ax=ax[i], dodge=True)

    # # Adding error bars for delta activations
    ax[i].set_xlabel("Muscles", fontsize=16)
    ax[i].tick_params(axis='x', labelrotation=45, labelsize=16)
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].set_ylabel("Peak Activation $\Delta$ (%)", fontsize=20)
    ax[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    ax[i].set_title(profile_names_paper[profile], fontsize=24)
    ax[i].tick_params(labelleft=True)

    handles, labels = ax[i].get_legend_handles_labels()

    ax[i].get_legend().remove()

    
fig.legend(handles, labels, fontsize=20, ncols=len(handles), bbox_to_anchor=(0.56, 1.02))  
plt.tight_layout()
plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 5))
fig.suptitle(f"Average PEAK muscle activation $\Delta$\nper subject and profile", fontsize=20, y=1.02)

df_all_deltas = pd.concat(all_deltas)

df_all_deltas.sort_values(by="Muscle", ascending=True, inplace=True)

In [ ]:
# Create a figure and axis for the plot
plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = Path("../paper_figures/") / f"peak_activation_deltas_per_subject.svg"

# Initialize a dictionary to store Mean Delta values for each profile across subjects
profile_deltas = {}

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_delta_activations = subject_data[subject][session]["peak_activation_deltas"]
        muscle_averages = mean_delta_activations[mean_delta_activations["Muscle"] == "Average"]

        muscle_averages.loc[:, "Profile"] = list(profile_peaks.values())

        # Plotting each subject's data
        axs.plot(muscle_averages["Profile"], muscle_averages["Mean Delta"], label=subject, marker='*', linewidth=3, markersize=10)
        
        # Accumulate Mean Delta values for averaging later
        for index, row in muscle_averages.iterrows():
            if row["Profile"] not in profile_deltas:
                profile_deltas[row["Profile"]] = []
            profile_deltas[row["Profile"]].append(row["Mean Delta"])

# Calculate and plot the average across subjects for each profile
average_deltas = {profile: np.mean(deltas) for profile, deltas in profile_deltas.items()}
profiles = profile_names_paper
average_values = list(average_deltas.values())

axs.set_xticks(list(profile_peaks.values()))

axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
axs.hlines(0, profile_peaks['peak_27%_62N'], profile_peaks['peak_77%_62N'], color='black', linestyle='--')
axs.set_xlabel("Peak Timing of Assistance (%)")
axs.set_ylabel("Peak Activation Delta\n(% of baseline peak)")
axs.set_ylim(-0.4, 0.4)
axs.legend(fontsize=16, title="Subjects", title_fontsize='16', loc='upper right')

plt.savefig(filename, format='svg', bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png", bbox_inches='tight')
plt.show()

In [ ]:
assisted_average_activations = pd.concat([subject_data[subject][session]['averaged_activations']['ASSISTED'][profile]['EMG']['mean'] for profile in profile_infos.keys()], axis=1).T.mean()

In [ ]:
def label_bars(ax: plt.axes, bars: list, texts: list, orient: Literal['v', 'h'] = 'v'):
    for i, bar in enumerate(bars):
        if orient == 'v':
            # For vertical bars, position the label above the bar
            bar_height = bar.get_height()
            bar_x = bar.get_x()
            bar_width = bar.get_width()
            center = bar_x + bar_width / 2
            ax.text(center, bar_height, texts[i], ha='center', va='bottom', fontsize=14)
        else:
            # For horizontal bars, position the label to the right of the bar
            bar_width = bar.get_width()
            bar_y = bar.get_y()
            bar_height = bar.get_height()
            center = bar_y + bar_height / 2
            # Determine the horizontal alignment based on the sign of the bar width
            ha = 'left' if bar_width >= 0 else 'right'
            # Offset the text from the end of the bar by a fraction of the bar width
            offset = bar_width * 0.01 if bar_width >= 0 else bar_width * 0.01 - 0.02
            ax.text(bar_width + offset, center, texts[i], ha=ha, va='center', fontsize=14)


In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 10))
fig.suptitle("Average muscle activation across all profiles", fontsize=20)

for i, subject in enumerate(subject_data.keys()):
    subject_df = df_all_means[df_all_means["Subject"] == subject].drop(columns=["Subject", "Session"])
    baseline_df = subject_df[subject_df["Profile"] == "BASELINE"].drop(columns=["Profile"])

    subject_df = subject_df[(subject_df["Profile"] != "BASELINE") & (subject_df["Muscle"] != "AVERAGE")]
    subject_df = subject_df.drop(columns='Profile').groupby('Muscle').mean().reset_index()
    subject_df['Group'] = 'Assisted'
    baseline_df['Group'] = 'Baseline'

    # Muscle activation averages by muscle and subject across profiles.
    muscle_avg_across_profiles = pd.concat([subject_df, baseline_df], ignore_index=True).sort_values(by="Group", ascending=False)
    assisted_df = muscle_avg_across_profiles[muscle_avg_across_profiles["Group"] == "Assisted"].set_index("Muscle")
    baseline_df = muscle_avg_across_profiles[muscle_avg_across_profiles["Group"] == "Baseline"].set_index("Muscle")
    
    sns.barplot(x="Muscle", y="Mean Activation", data=muscle_avg_across_profiles, hue='Group', ax=ax[i])


    percentage_change = (assisted_df['Mean Activation'] - baseline_df['Mean Activation']) / baseline_df['Mean Activation']
    percentage_change = percentage_change.reset_index()
    percentage_change.columns = ["Muscle", "Activation Delta"]
    percentage_change = percentage_change.values[:, 1]

    percentage_change = [f"{val*100:.0f}%" for val in percentage_change]


    bars = ax[i].patches[:-2]

    baseline_bars = bars[:len(bars)//2]
    assisted_bars = bars[len(bars)//2:]

    label_bars(ax[i], baseline_bars, len(bars)//2 * ["0%"])
    label_bars(ax[i], assisted_bars, percentage_change)
    
    ax[0].set_ylim(0, 0.5)
    ax[1].set_ylim(0, 0.7)
    ax[i].legend().remove()
    ax[i].set_title(f"Subject {subject}")
    ax[i].set_ylabel("Average Muscle Activation (% of MVIC)")
    ax[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None))
    handles, labels = ax[i].get_legend_handles_labels()

fig.legend(handles=handles, labels=labels, loc="upper right", fontsize=16, title="Condition", title_fontsize=16, bbox_to_anchor=(0.95, 1.02))
plt.tight_layout(rect=[0, 0.03, 1, 0.98])
plt.savefig("../paper_figures/assisted_vs_baseline_activation_change.svg", format='svg', bbox_inches='tight')
plt.savefig("../paper_figures/assisted_vs_baseline_activation_change.png", dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / "assisted_vs_baseline_activation_change.png", dpi=500, format="png", bbox_inches='tight')
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5), squeeze=False)  # Ensure axs is always 2D
fig.suptitle("Absolute Average Muscle Activation\nAssisted vs Baseline", fontsize=20)
plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"assisted_vs_baseline_subjects.svg"

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        df = subject_data[subject][session]["absolute_mean_activations"]
        assisted_mean = df[df["Muscle"] == "AVERAGE"][["Mean Activation", "STD Activation"]].mean()
        baseline_mean = df[df["Profile"] == "BASELINE"][["Mean Activation", "STD Activation"]].mean()
        
        # Adjusted sns.barplot to plot horizontally
        sns.barplot(y=["Assisted", "Baseline"], 
                    x=[assisted_mean["Mean Activation"], baseline_mean["Mean Activation"]],
                    orient='h', ax=axs[0, i], palette=[u'#3f88c5', u'#d72638'], hue=["Assisted", "Baseline"])  # Note the swapped x and y parameters
        
        percentage_change = (assisted_mean['Mean Activation'] - baseline_mean['Mean Activation']) / baseline_mean['Mean Activation']
        print(percentage_change)

        label_bars(ax=axs[0, i], bars=axs[0, i].patches, texts=[f"{percentage_change*100:.0f}%", "0%"], orient='h')
        
        # Adjust title, labels, and tick parameters for horizontal orientation
        axs[0, i].set_title(f"Subject {subject}", fontsize=20)
        axs[0, i].set_ylabel("Condition", fontsize=16)  # Now y-axis is "Condition"
        axs[0, i].set_xlabel("Mean Activation\n(% of MVIC)", fontsize=16)  # x-axis shows Mean Activation
        
        axs[0, i].tick_params(axis='y', labelsize=14)  # Adjust y-tick labels for "Condition"
        axs[0, i].tick_params(axis='x', labelsize=14)  # Adjust x-tick labels for Mean Activation
        
        axs[0, i].set_xlim(0, 0.5)  # Adjust x-limits to fit the data
        axs[0, i].xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None, decimals=0))  # Apply percent formatter to x-axis

plt.tight_layout()
plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Calculate correlation between applied force profile and simulated profile

In [ ]:
from matplotlib.patches import FancyArrowPatch

subject = "MIH02"
session = "December_20"

data = subject_data[subject][session]

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"osim_27_47.svg"

prof_27 = "peak_27%_62N"
prof_47 = "peak_47%_62N"

line_names = ['27% peak timing', '47% peak timing']
n_subjects = len(subject_data)  # Total number of subjects

# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
fig, axs = plt.subplots(figsize=(6, 4), constrained_layout=True)
for profile, name in zip(target_splines, profile_names_paper.values()):
    force_Y = profile.force_Y
    sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
    sim_profile_interp.index = profile.Percentage

    # Plot original profiles
    axs.plot(profile.Percentage, force_Y, label=name, linestyle='-', alpha=0.7, linewidth=3)
    # Remove y-axis ticks and labels
    axs.xaxis.set_major_formatter(mticker.PercentFormatter())

axs.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='OpenSim Assistance', linestyle='--', color='black', alpha=0.7, linewidth=3)

axs.grid(False)
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

axs.spines['left'].set_visible(False)  # Adjust thickness of the arrow
axs.spines['bottom'].set_visible(False)  # Adjust thickness of the arrow

# Make x-axis and y-axis arrows
axs.spines['left'].set_linewidth(2)
axs.spines['bottom'].set_linewidth(1)  # Adjust thickness of the arrow
axs.spines['bottom'].set_capstyle('projecting')  # Arrow style
offset = 0.03

# Add padding to figure

axs.add_patch(FancyArrowPatch((offset, offset), (1 - offset, offset),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))

axs.add_patch(FancyArrowPatch((offset, offset), (offset, 1),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))
# add padding to figure
# Adjust ticks
axs.xaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
axs.yaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
# Set labels
axs.set_xlabel("Sit-to-Stand (%)")
axs.set_ylabel("Assistance (N)")

# Customize legend
axs.legend(loc=(0.04, 1), frameon=False, fontsize=8)

plt.savefig("../paper_figures/osim_27_47.svg", format='svg', bbox_inches='tight')
plt.savefig(filename, format='svg', bbox_inches='tight')
plt.show()

In [ ]:
n_subjects = len(subject_data)  # Total number of subjects
# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
filename = Path("../paper_figures/") / f"profile_correlations.svg"

fig, axs = plt.subplots(3, n_subjects, figsize=(10 * n_subjects, 18), constrained_layout=True)
fig.suptitle("Correlation and Sum of Correlations Across Subjects", fontsize=16)

if n_subjects == 1:
    axs = axs[:, np.newaxis]  # Ensure axs is 2D even for a single subplot

for subject_idx, (subject, sessions) in enumerate(subject_data.items(), start=1):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])
    colors.__next__() # Reset colors for each subject
    ax_profile = axs[0, subject_idx - 1]  # Axes for the original profiles
    ax_corr = axs[1, subject_idx - 1]  # Axes for the correlation plot
    ax_sum = axs[2, subject_idx - 1]  # Axes for the sum of correlations bar plot
    session_name = next(iter(sessions))  # Assuming there's only one session per subject
    data = sessions[session_name]

    ax_profile.set_title(f"Subject: {subject}\nSession: {session_name}\nOriginal Profiles", fontsize=14)
    ax_corr.set_title("Correlation Plots", fontsize=14)
    total_correlation = []  # List to store sum of correlations for each profile
    
    for profile in data["average_profile_dfs"]["ASSISTED"].keys():
        force_Y = data["average_profile_dfs"]["ASSISTED"][profile]["MOTOR_DATA"]["mean"].F_Y
        sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
        sim_profile_interp.index = force_Y.index

        # Plot original profiles
        ax_profile.plot(force_Y.index, force_Y, label=f'{profile_names_paper[profile]} Experimental', linestyle='-', alpha=0.7, linewidth=4)
        ax_profile.set_xlabel("Percentage of STS", fontsize=12)
        ax_profile.set_ylabel("Force", fontsize=12)
        ax_profile.xaxis.set_major_formatter(mticker.PercentFormatter())
        ax_profile.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}N'))
        
        # Plot simulation profile as a dashed black line

        # Calculate correlation
        corr = np.correlate(force_Y, sim_profile_interp["force_Y"], 'same')
        total_correlation.append(np.sum(corr))  # Sum of correlation for the current profile

        # Plot correlation for the current profile on the second row
        ax_corr.plot(force_Y.index, corr, label=profile_names_paper[profile], linewidth=4)
        ax_corr.set_xlabel("Percentage of STS", fontsize=12)
        ax_corr.xaxis.set_major_formatter(mticker.PercentFormatter())

    ax_profile.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='Simulation profile', linestyle='--', color='black', alpha=0.7, linewidth=4)
    ax_profile.legend(loc='upper right', fontsize=8)  # Add legend to differentiate profiles
    ax_corr.legend(loc='best', fontsize=8)  # Add legend to differentiate profiles
    max_corr_index = np.argmax(total_correlation)  # Index of the profile with the highest sum of correlation
    
    # Bar plot for the sum of correlations on the third row
    bars = ax_sum.bar(range(len(total_correlation)), total_correlation, tick_label=list(profile_peaks.values()))
    bars[max_corr_index].set_color(colors.__next__())  # Highlight the max with red color

    ax_sum.set_title("Total Correlation by Profile", fontsize=14)
    ax_sum.set_ylabel("Sum of Correlation", fontsize=12)
    ax_sum.set_xlabel("Peak timing (%)", fontsize=12)

plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 5))
fig.suptitle(f"Average PEAK muscle activation $\Delta$\nper subject and profile", fontsize=20, y=1.02)

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = Path("../paper_figures/") / f"peak_activation_deltas_per_subject.svg"

# Initialize a dictionary to store Mean Delta values for each profile across subjects
profile_deltas = {}

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_delta_activations = subject_data[subject][session]["peak_activation_deltas"]
        muscle_averages = mean_delta_activations[mean_delta_activations["Muscle"] == "Average"]

        muscle_averages.loc[:, "Profile"] = list(profile_peaks.values())

        # Plotting each subject's data
        axs.plot(muscle_averages["Profile"], muscle_averages["Mean Delta"], label=subject, marker='*', linewidth=3, markersize=10)
        
        # Accumulate Mean Delta values for averaging later
        for index, row in muscle_averages.iterrows():
            if row["Profile"] not in profile_deltas:
                profile_deltas[row["Profile"]] = []
            profile_deltas[row["Profile"]].append(row["Mean Delta"])

# Calculate and plot the average across subjects for each profile
average_deltas = {profile: np.mean(deltas) for profile, deltas in profile_deltas.items()}
profiles = profile_names_paper
average_values = list(average_deltas.values())

axs.set_xticks(list(profile_peaks.values()))

axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
axs.hlines(0, profile_peaks['peak_27%_62N'], profile_peaks['peak_77%_62N'], color='black', linestyle='--')
axs.set_xlabel("Peak Timing of Assistance (%)")
axs.set_ylabel("Peak Activation Delta\n(% of baseline peak)")
axs.set_ylim(-0.4, 0.4)
axs.legend(fontsize=16, title="Subjects", title_fontsize='16', loc='upper right')

plt.savefig(filename, format='svg', bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png", bbox_inches='tight')
plt.show()

In [ ]:
assisted_average_activations = pd.concat([subject_data[subject][session]['averaged_activations']['ASSISTED'][profile]['EMG']['mean'] for profile in profile_infos.keys()], axis=1).T.mean()

In [ ]:
def label_bars(ax: plt.axes, bars: list, texts: list, orient: Literal['v', 'h'] = 'v'):
    for i, bar in enumerate(bars):
        if orient == 'v':
            # For vertical bars, position the label above the bar
            bar_height = bar.get_height()
            bar_x = bar.get_x()
            bar_width = bar.get_width()
            center = bar_x + bar_width / 2
            ax.text(center, bar_height, texts[i], ha='center', va='bottom', fontsize=14)
        else:
            # For horizontal bars, position the label to the right of the bar
            bar_width = bar.get_width()
            bar_y = bar.get_y()
            bar_height = bar.get_height()
            center = bar_y + bar_height / 2
            # Determine the horizontal alignment based on the sign of the bar width
            ha = 'left' if bar_width >= 0 else 'right'
            # Offset the text from the end of the bar by a fraction of the bar width
            offset = bar_width * 0.01 if bar_width >= 0 else bar_width * 0.01 - 0.02
            ax.text(bar_width + offset, center, texts[i], ha=ha, va='center', fontsize=14)


In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 10))
fig.suptitle("Average muscle activation across all profiles", fontsize=20)

for i, subject in enumerate(subject_data.keys()):
    subject_df = df_all_means[df_all_means["Subject"] == subject].drop(columns=["Subject", "Session"])
    baseline_df = subject_df[subject_df["Profile"] == "BASELINE"].drop(columns=["Profile"])

    subject_df = subject_df[(subject_df["Profile"] != "BASELINE") & (subject_df["Muscle"] != "AVERAGE")]
    subject_df = subject_df.drop(columns='Profile').groupby('Muscle').mean().reset_index()
    subject_df['Group'] = 'Assisted'
    baseline_df['Group'] = 'Baseline'

    # Muscle activation averages by muscle and subject across profiles.
    muscle_avg_across_profiles = pd.concat([subject_df, baseline_df], ignore_index=True).sort_values(by="Group", ascending=False)
    assisted_df = muscle_avg_across_profiles[muscle_avg_across_profiles["Group"] == "Assisted"].set_index("Muscle")
    baseline_df = muscle_avg_across_profiles[muscle_avg_across_profiles["Group"] == "Baseline"].set_index("Muscle")
    
    sns.barplot(x="Muscle", y="Mean Activation", data=muscle_avg_across_profiles, hue='Group', ax=ax[i])


    percentage_change = (assisted_df['Mean Activation'] - baseline_df['Mean Activation']) / baseline_df['Mean Activation']
    percentage_change = percentage_change.reset_index()
    percentage_change.columns = ["Muscle", "Activation Delta"]
    percentage_change = percentage_change.values[:, 1]

    percentage_change = [f"{val*100:.0f}%" for val in percentage_change]


    bars = ax[i].patches[:-2]

    baseline_bars = bars[:len(bars)//2]
    assisted_bars = bars[len(bars)//2:]

    label_bars(ax[i], baseline_bars, len(bars)//2 * ["0%"])
    label_bars(ax[i], assisted_bars, percentage_change)
    
    ax[0].set_ylim(0, 0.5)
    ax[1].set_ylim(0, 0.7)
    ax[i].legend().remove()
    ax[i].set_title(f"Subject {subject}")
    ax[i].set_ylabel("Average Muscle Activation (% of MVIC)")
    ax[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None))
    handles, labels = ax[i].get_legend_handles_labels()

fig.legend(handles=handles, labels=labels, loc="upper right", fontsize=16, title="Condition", title_fontsize=16, bbox_to_anchor=(0.95, 1.02))
plt.tight_layout(rect=[0, 0.03, 1, 0.98])
plt.savefig("../paper_figures/assisted_vs_baseline_activation_change.svg", format='svg', bbox_inches='tight')
plt.savefig("../paper_figures/assisted_vs_baseline_activation_change.png", dpi=500, format='png', bbox_inches='tight')
plt.savefig(latex_figure_dir / "emg_plots" / "assisted_vs_baseline_activation_change.png", dpi=500, format="png", bbox_inches='tight')
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5), squeeze=False)  # Ensure axs is always 2D
fig.suptitle("Absolute Average Muscle Activation\nAssisted vs Baseline", fontsize=20)
plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"assisted_vs_baseline_subjects.svg"

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        df = subject_data[subject][session]["absolute_mean_activations"]
        assisted_mean = df[df["Muscle"] == "AVERAGE"][["Mean Activation", "STD Activation"]].mean()
        baseline_mean = df[df["Profile"] == "BASELINE"][["Mean Activation", "STD Activation"]].mean()
        
        # Adjusted sns.barplot to plot horizontally
        sns.barplot(y=["Assisted", "Baseline"], 
                    x=[assisted_mean["Mean Activation"], baseline_mean["Mean Activation"]],
                    orient='h', ax=axs[0, i], palette=[u'#3f88c5', u'#d72638'], hue=["Assisted", "Baseline"])  # Note the swapped x and y parameters
        
        percentage_change = (assisted_mean['Mean Activation'] - baseline_mean['Mean Activation']) / baseline_mean['Mean Activation']
        print(percentage_change)

        label_bars(ax=axs[0, i], bars=axs[0, i].patches, texts=[f"{percentage_change*100:.0f}%", "0%"], orient='h')
        
        # Adjust title, labels, and tick parameters for horizontal orientation
        axs[0, i].set_title(f"Subject {subject}", fontsize=20)
        axs[0, i].set_ylabel("Condition", fontsize=16)  # Now y-axis is "Condition"
        axs[0, i].set_xlabel("Mean Activation\n(% of MVIC)", fontsize=16)  # x-axis shows Mean Activation
        
        axs[0, i].tick_params(axis='y', labelsize=14)  # Adjust y-tick labels for "Condition"
        axs[0, i].tick_params(axis='x', labelsize=14)  # Adjust x-tick labels for Mean Activation
        
        axs[0, i].set_xlim(0, 0.5)  # Adjust x-limits to fit the data
        axs[0, i].xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None, decimals=0))  # Apply percent formatter to x-axis

plt.tight_layout()
plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()

### Calculate correlation between applied force profile and simulated profile

In [ ]:
from matplotlib.patches import FancyArrowPatch

subject = "MIH02"
session = "December_20"

data = subject_data[subject][session]

plot_dir = subject_dirs[subject][session]["plot_dir"]
filename = plot_dir / f"osim_27_47.svg"

prof_27 = "peak_27%_62N"
prof_47 = "peak_47%_62N"

line_names = ['27% peak timing', '47% peak timing']
n_subjects = len(subject_data)  # Total number of subjects

# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
fig, axs = plt.subplots(figsize=(6, 4), constrained_layout=True)
for profile, name in zip(target_splines, profile_names_paper.values()):
    force_Y = profile.force_Y
    sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
    sim_profile_interp.index = profile.Percentage

    # Plot original profiles
    axs.plot(profile.Percentage, force_Y, label=name, linestyle='-', alpha=0.7, linewidth=3)
    # Remove y-axis ticks and labels
    axs.xaxis.set_major_formatter(mticker.PercentFormatter())

axs.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='OpenSim Assistance', linestyle='--', color='black', alpha=0.7, linewidth=3)

axs.grid(False)
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

axs.spines['left'].set_visible(False)  # Adjust thickness of the arrow
axs.spines['bottom'].set_visible(False)  # Adjust thickness of the arrow

# Make x-axis and y-axis arrows
axs.spines['left'].set_linewidth(2)
axs.spines['bottom'].set_linewidth(1)  # Adjust thickness of the arrow
axs.spines['bottom'].set_capstyle('projecting')  # Arrow style
offset = 0.03

# Add padding to figure

axs.add_patch(FancyArrowPatch((offset, offset), (1 - offset, offset),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))

axs.add_patch(FancyArrowPatch((offset, offset), (offset, 1),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))
# add padding to figure
# Adjust ticks
axs.xaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
axs.yaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
# Set labels
axs.set_xlabel("Sit-to-Stand (%)")
axs.set_ylabel("Assistance (N)")

# Customize legend
axs.legend(loc=(0.04, 1), frameon=False, fontsize=8)

plt.savefig("../paper_figures/osim_27_47.svg", format='svg', bbox_inches='tight')
plt.savefig(filename, format='svg', bbox_inches='tight')
plt.show()

In [ ]:
n_subjects = len(subject_data)  # Total number of subjects
# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
filename = Path("../paper_figures/") / f"profile_correlations.svg"

fig, axs = plt.subplots(3, n_subjects, figsize=(10 * n_subjects, 18), constrained_layout=True)
fig.suptitle("Correlation and Sum of Correlations Across Subjects", fontsize=16)

if n_subjects == 1:
    axs = axs[:, np.newaxis]  # Ensure axs is 2D even for a single subplot

for subject_idx, (subject, sessions) in enumerate(subject_data.items(), start=1):
    colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])
    colors.__next__() # Reset colors for each subject
    ax_profile = axs[0, subject_idx - 1]  # Axes for the original profiles
    ax_corr = axs[1, subject_idx - 1]  # Axes for the correlation plot
    ax_sum = axs[2, subject_idx - 1]  # Axes for the sum of correlations bar plot
    session_name = next(iter(sessions))  # Assuming there's only one session per subject
    data = sessions[session_name]

    ax_profile.set_title(f"Subject: {subject}\nSession: {session_name}\nOriginal Profiles", fontsize=14)
    ax_corr.set_title("Correlation Plots", fontsize=14)
    total_correlation = []  # List to store sum of correlations for each profile
    
    for profile in data["average_profile_dfs"]["ASSISTED"].keys():
        force_Y = data["average_profile_dfs"]["ASSISTED"][profile]["MOTOR_DATA"]["mean"].F_Y
        sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
        sim_profile_interp.index = force_Y.index

        # Plot original profiles
        ax_profile.plot(force_Y.index, force_Y, label=f'{profile_names_paper[profile]} Experimental', linestyle='-', alpha=0.7, linewidth=4)
        ax_profile.set_xlabel("Percentage of STS", fontsize=12)
        ax_profile.set_ylabel("Force", fontsize=12)
        ax_profile.xaxis.set_major_formatter(mticker.PercentFormatter())
        ax_profile.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}N'))
        
        # Plot simulation profile as a dashed black line

        # Calculate correlation
        corr = np.correlate(force_Y, sim_profile_interp["force_Y"], 'same')
        total_correlation.append(np.sum(corr))  # Sum of correlation for the current profile

        # Plot correlation for the current profile on the second row
        ax_corr.plot(force_Y.index, corr, label=profile_names_paper[profile], linewidth=4)
        ax_corr.set_xlabel("Percentage of STS", fontsize=12)
        ax_corr.xaxis.set_major_formatter(mticker.PercentFormatter())

    ax_profile.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='Simulation profile', linestyle='--', color='black', alpha=0.7, linewidth=4)
    ax_profile.legend(loc='upper right', fontsize=8)  # Add legend to differentiate profiles
    ax_corr.legend(loc='best', fontsize=8)  # Add legend to differentiate profiles
    max_corr_index = np.argmax(total_correlation)  # Index of the profile with the highest sum of correlation
    
    # Bar plot for the sum of correlations on the third row
    bars = ax_sum.bar(range(len(total_correlation)), total_correlation, tick_label=list(profile_peaks.values()))
    bars[max_corr_index].set_color(colors.__next__())  # Highlight the max with red color

    ax_sum.set_title("Total Correlation by Profile", fontsize=14)
    ax_sum.set_ylabel("Sum of Correlation", fontsize=12)
    ax_sum.set_xlabel("Peak timing (%)", fontsize=12)

plt.savefig(filename, format='svg')
plt.savefig(filename.with_suffix('.png'), dpi=500, format='png')
plt.savefig(latex_figure_dir / "emg_plots" / filename.with_suffix('.png').name, dpi=500, format="png")
plt.show()